# KATL Final Approach Simulation
### ! conda activate tracon

In [1]:
import numpy as np
import pandas as pd
import math
import pickle

# save pickle full state
# st = np.random.get_state()
# with open("state1.pkl", "wb") as f:
#     pickle.dump(st, f)


# # load pickle full state
# with open("state1.pkl", "rb") as f:
#     state = pickle.load(f)

# # 2. Restore NumPy's RNG to that state
# np.random.set_state(state)
SEED = 123
# ==========================================
# 1. PARAMETERS & CONSTANTS
# ==========================================

# Simulation Constraints
R_TURN_NM = 6.0       # Radius of RF turn (nmi)
T_SEP_SEC = 66.0       # Minimum separation (seconds)
T_MAX_SEC = 3600.0     # Simulation duration (1 hour)

# Unit Conversions
KM_TO_NM = 0.539957    # 1 Kilometer = 0.539957 Nautical Miles
R_EARTH_KM = 6371.0    # Radius of Earth in KM

# Reference Point: KATL Runway 9L Threshold (Origin 0,0)
REF_POINT = {'lat': 33.634667, 'lon': -84.448000}

# Fix Definitions (Lat/Lon)
# Note: VINII is included here to calculate its relative X/Y position
raw_fixes = {
    'VINII': {'lat': 33.634650, 'lon': -84.549842, 'type': 'FAF', 'flow': 'FAF'},
    'DALAS': {'lat': 33.952250, 'lon': -84.848022, 'type': 'Corner', 'flow': 'NorthWest'},
    'LOGEN': {'lat': 33.988050, 'lon': -84.056786, 'type': 'Corner', 'flow': 'NorthEast'},
    'HUSKY': {'lat': 33.330458, 'lon': -83.980208, 'type': 'Corner', 'flow': 'SouthEast'},
    'TIROE': {'lat': 33.306453, 'lon': -84.866031, 'type': 'Corner', 'flow': 'SouthWest'}
}

# ==========================================
# 2. GEOMETRY FUNCTIONS (Great Circle)
# ==========================================

def get_distance_and_bearing(lat1, lon1, lat2, lon2):
    """
    Calculates Great Circle distance (km) and initial bearing (degrees)
    between two points.
    """
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    # --- Haversine Formula for Distance ---
    a = math.sin(dphi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    dist_km = R_EARTH_KM * c
    
    # --- Bearing Formula ---
    y = math.sin(dlambda) * math.cos(phi2)
    x = math.cos(phi1) * math.sin(phi2) - math.sin(phi1) * math.cos(phi2) * math.cos(dlambda)
    bearing_rad = math.atan2(y, x)
    bearing_deg = (math.degrees(bearing_rad) + 360) % 360
    
    return dist_km, bearing_deg

def geo_to_cartesian(target_lat, target_lon, ref_lat, ref_lon):
    """
    Converts target Lat/Lon to X (East) / Y (North) in Nautical Miles
    relative to the reference point (0,0).
    """
    # 1. Get Distance (km) and Bearing (deg)
    dist_km, bearing_deg = get_distance_and_bearing(ref_lat, ref_lon, target_lat, target_lon)
    
    # 2. Convert to NM
    dist_nm = dist_km * KM_TO_NM
    
    # 3. Project to Cartesian (Standard Math: 0 deg = East, but Map: 0 deg = North)
    # Map Bearing: 0 is North (Y+), 90 is East (X+)
    bearing_rad = math.radians(bearing_deg)
    
    x_nm = dist_nm * math.sin(bearing_rad) # Sin gives East component for Map Bearing
    y_nm = dist_nm * math.cos(bearing_rad) # Cos gives North component for Map Bearing
    
    return x_nm, y_nm

# ==========================================
# 3. INITIALIZE GEOMETRY
# ==========================================

processed_fixes = {}

for name, data in raw_fixes.items():
    x, y = geo_to_cartesian(data['lat'], data['lon'], REF_POINT['lat'], REF_POINT['lon'])
    
    processed_fixes[name] = data.copy()
    processed_fixes[name]['x'] = x
    processed_fixes[name]['y'] = y
    processed_fixes[name]['dist (nmi)'] = np.sqrt(x**2 + y**2)
    processed_fixes[name]['flow'] = data['flow']

print("Geometry Setup Complete.")
print(f"Origin (Runway 9L Threshold): (0.0, 0.0) nm")
print(f"FAF (VINII) Calculated Location: ({processed_fixes['VINII']['x']:.4f}, {processed_fixes['VINII']['y']:.4f}) nm")
print(f"\n⚠️ IMPORTANT: T_SEP_SEC = {T_SEP_SEC} seconds (Minimum separation requirement)")
pd.DataFrame(processed_fixes).T[['type', 'x', 'y', 'dist (nmi)', 'flow']] # Display Table

Geometry Setup Complete.
Origin (Runway 9L Threshold): (0.0, 0.0) nm
FAF (VINII) Calculated Location: (-5.0910, 0.0015) nm

⚠️ IMPORTANT: T_SEP_SEC = 66.0 seconds (Minimum separation requirement)


,type,x,y,dist (nmi),flow
VINII,FAF,-5.090967,0.001485,5.090968,FAF
DALAS,Corner,-19.922658,19.106464,27.603791,NorthWest
LOGEN,Corner,19.475811,21.254227,28.827927,NorthEast
HUSKY,Corner,23.466697,-18.211933,29.704551,SouthEast
TIROE,Corner,-20.976252,-19.663862,28.751881,SouthWest


In [2]:
# ==========================================
# 4. ARRIVAL GENERATION FUNCTIONS
# ==========================================

def generate_shifted_poisson_times(lambda_ph, t_sep, t_max):
    """
    Generates a list of arrival times (seconds) based on Shifted Poisson Process.
    Delta_t = T_sep + Exp(1/Lambda)
    """
    lambda_per_sec = lambda_ph / 3600.0
    arrivals = []
    t = 0.0
    
    while t < t_max:
        # Stochastic component: Exponential distribution
        stochastic_wait = np.random.exponential(1.0 / lambda_per_sec)
        
        # Total interval: Hard separation + Stochastic
        inter_arrival = t_sep + stochastic_wait
        
        t += inter_arrival
        if t > t_max:
            break
        arrivals.append(t)
        
    return arrivals

def generate_scenario(seed=None):
    np.random.seed(seed)
    all_flights = []
    
    # Retrieve FAF location for Logic checks
    faf_x = processed_fixes['VINII']['x']
    faf_y = processed_fixes['VINII']['y']

    # Iterate only through the 4 corners (exclude VINII from being a start point)
    corners = [k for k, v in processed_fixes.items() if v['type'] == 'Corner']
    
    print(f"{'Corner':<10} | {'Lambda (arr/hr)':<15} | {'Count'}")
    print("-" * 40)

    for name in corners:
        fix_data = processed_fixes[name]
        
        # 1. Sample Demand (Log-Uniform between 5 and 45 aircraft/hr)
        # This ensures we test both low and high traffic scenarios
        log_lambda = np.random.uniform(np.log(5), np.log(15))
        lambda_s = np.exp(log_lambda)
        
        # 2. Generate Times
        times = generate_shifted_poisson_times(lambda_s, T_SEP_SEC, T_MAX_SEC)
        
        # 3. Determine Geometry Logic
        # North/South Logic (for turn direction center calculation later)
        is_north = fix_data['y'] > faf_y
        
        # LongArc Logic: 
        # If Fix X > FAF X, plane is "downwind" relative to FAF -> LongArc
        is_long_arc = 1 if fix_data['x'] > faf_x else 0
        
        print(f"{name:<10} | {lambda_s:>15.2f} | {len(times)}")
        
        # 4. Store Data
        for t in times:
            all_flights.append({
                'entry_time': t,
                'corner': fix_data['flow'],
                'enter_fix_name': name,
                # Entry State
                'x_entry': fix_data['x'],
                'y_entry': fix_data['y'],
                # Target State (FAF)
                'x_faf': faf_x,
                'y_faf': faf_y,
                # Optimization Flags
                'r_turn': R_TURN_NM,
                'is_north': is_north,
                'long_arc': is_long_arc
            })
            
    # Create DataFrame and sort by time
    df = pd.DataFrame(all_flights).sort_values('entry_time').reset_index(drop=True)
    df['aircraft_id'] = df.index + 1
    
    return df

# ==========================================
# 5. EXECUTE GENERATION
# ==========================================

df_arrivals = generate_scenario(seed=SEED)

print("\nSimulation Generated.")
print(f"Total Aircraft: {len(df_arrivals)}")
print("\nFirst 10 Arrivals:")
display(df_arrivals)

Corner     | Lambda (arr/hr) | Count
----------------------------------------
DALAS      |           10.75 | 8
LOGEN      |            7.29 | 9
HUSKY      |           12.71 | 14
TIROE      |           13.34 | 9

Simulation Generated.
Total Aircraft: 40

First 10 Arrivals:


,entry_time,corner,enter_fix_name,x_entry,y_entry,x_faf,y_faf,r_turn,is_north,long_arc,aircraft_id
0,178.914341,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,1
1,331.102014,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,2
2,431.010455,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,3
3,665.574532,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,4
4,710.879026,NorthEast,LOGEN,19.475811,21.254227,-5.090967,0.001485,6.0,True,1,5
5,764.391897,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,6
6,844.508337,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,7
7,1061.964834,NorthEast,LOGEN,19.475811,21.254227,-5.090967,0.001485,6.0,True,1,8
8,1098.527750,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,9
9,1157.370978,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,10


# Pyomo Optimization

# Setup I: Optimize $d_i$ Only with FCFS (Fixed Sequence)

In [3]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# ==========================================
# 6. OPTIMIZATION MODEL (Fixed Sequence)
# ==========================================

def solve_schedule_optimization(df_arrivals, debug=False):
    """
    Builds and solves the NLP model to find optimal d_i (extension)
    to minimize makespan while satisfying T_sep.
    """
    
    # --- A. Setup Model ---
    m = pyo.ConcreteModel()
    
    # Indices (Aircraft IDs)
    # We assume the dataframe is already sorted by entry time (FCFS)
    flight_ids = df_arrivals['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Parameters (Fixed Data from DF)
    # We use dictionaries to map ID -> Value for Pyomo
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x_entry = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y_entry = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_is_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long_arc = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    # Global Geometry Constants
    r = R_TURN_NM
    x_faf = processed_fixes['VINII']['x']
    y_faf = processed_fixes['VINII']['y']
    
    # Speeds (Knots) - Fixed for this formulation
    # v_L: Tangent leg, v_theta: Turn, v_f: Final straight
    v_L = 210.0
    v_theta = 180.0
    v_f = 150.0
    
    # Bounds for d_i (The extension leg)
    # Min 0.0, Max 20.0 nm (approx 37km extension limit)
    d_min, d_max = 0.0, 20.0 
    
    # --- B. Decision Variables ---
    # d[i]: The extension distance from FAF (nautical miles)
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(d_min, d_max), initialize=0.0)
    
    # t[i]: The arrival time at FAF (seconds)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=lambda m, i: p_tau[i] + 500)

    # --- C. Geometry Expressions (The Math) ---
    # We define these as expressions so Pyomo can calculate derivatives
    
    def calc_travel_time_expr(m, i):
        # 1. Determine Center of Turn C0 based on d[i]
        # If North Arrival: Center is (X_faf - d, Y_faf + r)
        # If South Arrival: Center is (X_faf - d, Y_faf - r)
        # We use standard python if because 'p_is_north' is a parameter (constant)
        
        y_center_offset = r if p_is_north[i] else -r
        
        # Coordinates of Center C0
        Cx = x_faf - m.d[i]
        Cy = y_faf + y_center_offset
        
        # Coordinates of Projected Center C0_prime
        Cx_prime = x_faf - m.d[i]
        Cy_prime = y_faf
        
        # 2. Calculate Distances d0 and d0_prime
        # Distance from Entry (Px, Py) to Center C0
        # pyo.sqrt is required for variables
        d0_sq = (p_x_entry[i] - Cx)**2 + (p_y_entry[i] - Cy)**2
        d0 = pyo.sqrt(d0_sq)
        
        # Distance from Entry to Projected Center C0_prime
        d0_prime_sq = (p_x_entry[i] - Cx_prime)**2 + (p_y_entry[i] - Cy_prime)**2
        
        # 3. Tangent Distance d_L (Pythagoras)
        # d_L = sqrt(d0^2 - r^2)
        # Small epsilon 1e-6 added to prevent sqrt(0) error if d0=r
        d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
        
        # 4. Angle Calculation (Thetas)
        # theta1 = acos((r^2 + d0^2 - d_L^2) / (2*r*d0)) -> Simplifies to acos(r/d0)
        # theta2 = acos((r^2 + d0^2 - d0_prime^2) / (2*r*d0))
        
        # We clamp inputs to acos to [-1, 1] to prevent numerical errors
        term1 = r / d0
        term2 = (r**2 + d0_sq - d0_prime_sq) / (2 * r * d0)
        
        theta1 = pyo.acos(term1)
        theta2 = pyo.acos(term2)
        
        # 5. Total Turn Angle theta
        if p_long_arc[i]:
            # Downwind: 2*pi - (t1 + t2)
            theta_radians = 2*3.14159 - (theta1 + theta2)
        else:
            # Base/Straight: t2 - t1
            theta_radians = theta2 - theta1
            
        # 6. Distances
        dist_turn = r * theta_radians
        dist_final = m.d[i] # The straight-in extension
        
        # 7. Total Time (in Seconds)
        # Distance (nm) / Speed (knots) * 3600 (sec/hr)
        time_L = (d_L / v_L) * 3600
        time_turn = (dist_turn / v_theta) * 3600
        time_final = (dist_final / v_f) * 3600
        
        return p_tau[i] + time_L + time_turn + time_final

    # --- D. Constraints ---
    
    # 1. Link Physics to Time Variable
    def physics_rule(m, i):
        return m.t[i] == calc_travel_time_expr(m, i)
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # 2. Separation Constraint (Sequence Preserved)
    def separation_rule(m, i):
        # Skip the first aircraft
        prev_i = i - 1
        if prev_i not in m.I:
            return pyo.Constraint.Skip
        return m.t[i] >= m.t[prev_i] + T_SEP_SEC
    m.c_separation = pyo.Constraint(m.I, rule=separation_rule)
    
    # --- E. Objective ---
    # Minimize the arrival time of the LAST aircraft (Makespan)
    last_id = flight_ids[-1]
    m.obj = pyo.Objective(expr=m.t[last_id], sense=pyo.minimize)
    
    # --- F. Solve ---
    solver = SolverFactory('ipopt')
    solver.options['max_iter'] = 10000
    if not debug:
        solver.options['print_level'] = 0 # Suppress solver text
        
    result = solver.solve(m, tee=debug)
    
    # --- G. Extract Results ---
    results = []
    for i in m.I:
        results.append({
            'aircraft_id': i,
            'optimized_d_i': pyo.value(m.d[i]),
            'arrival_time_faf': pyo.value(m.t[i]),
            'scheduled_entry': p_tau[i]
        })
        
    return pd.DataFrame(results)

# ==========================================
# 7. RUN OPTIMIZATION
# ==========================================

print("Optimizing Schedule...")
df_opt_results = solve_schedule_optimization(df_arrivals, debug=True)

# Merge results back with original info for viewing
df_final = pd.merge(df_arrivals, df_opt_results, on='aircraft_id')

# Calculate Delay (Arrival Time - Minimum Possible Time)
# Note: This is simplified; true delay is typically (Actual - Scheduled)
# Here we just look at the timeline.
df_final['transit_time'] = df_final['arrival_time_faf'] - df_final['entry_time']

print("\nOptimization Complete.")
display(df_final[['aircraft_id', 'corner', 'entry_time', 'optimized_d_i', 'arrival_time_faf', 'transit_time']])

Optimizing Schedule...
Ipopt 3.14.19: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       80
Number of nonzeros in inequality constraint Jacobian.:       78
Number of nonzeros in Lagrangian Hessian.............:       40

Total number of variables............................:       80
                     variables with only lower bounds:       40
                variables with lower and upper bounds:       40
                     variables with only upper bounds:        0
Total number of equality constraints..

,aircraft_id,corner,entry_time,optimized_d_i,arrival_time_faf,transit_time
0,1,NorthWest,178.914341,1.219942e+01,897.031796,718.117455
1,2,NorthWest,331.102014,1.472811e+01,1128.924928,797.822914
2,3,SouthEast,431.010455,2.704081e-01,1316.312162,885.301707
3,4,NorthWest,665.574532,1.621384e+01,1488.448061,822.873529
4,5,NorthEast,710.879026,1.093487e+00,1575.333920,864.454894
5,6,SouthEast,764.391897,2.797896e-01,1650.077204,885.685307
6,7,SouthWest,844.508337,1.884986e+01,1729.505907,884.997571
7,8,NorthEast,1061.964834,4.508423e-02,1884.105760,822.140926
8,9,SouthWest,1098.527750,1.702984e+01,1951.442103,852.914353
9,10,NorthWest,1157.370978,1.837167e+01,2018.809712,861.438733


## Introducing Slack Variables for High Density Scenarios

In [4]:
# ==========================================
# 6. OPTIMIZATION MODEL (Fixed Speed + Slack)
# ==========================================

def solve_schedule_optimization_fixed_speed_robust(df_arrivals, debug=False):
    """
    Implements Section 2.2: Optimize d_i only.
    Includes Slack Variables to handle infeasibility during high traffic.
    """
    
    m = pyo.ConcreteModel()
    
    # --- A. Sets & Parameters ---
    flight_ids = df_arrivals['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Dictionaries for fast lookup
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x_entry = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y_entry = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_is_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long_arc = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    # Geometry Constants
    r = R_TURN_NM
    x_faf = processed_fixes['VINII']['x']
    y_faf = processed_fixes['VINII']['y']
    
    # FIXED SPEEDS (Knots) - Constants, not Variables
    v_L_const = 210.0
    v_theta_const = 180.0
    v_f_const = 150.0
    
    # --- B. Decision Variables ---
    
    # 1. Path Extension (The only control variable)
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    
    # 2. Arrival Time (Dependent variable)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=lambda m, i: p_tau[i] + 500)

    # 3. Slack Variable (The "Safety Valve")
    # Represents seconds of violation allowed to prevent solver crash
    m.slack = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=0.0)

    # --- C. Geometry Expressions ---
    def calc_travel_time_expr(m, i):
        # Center Calculation
        y_center_offset = r if p_is_north[i] else -r
        Cx = x_faf - m.d[i]
        Cy = y_faf + y_center_offset
        Cx_prime = x_faf - m.d[i]
        Cy_prime = y_faf
        
        # Distances (d0, d0_prime)
        d0_sq = (p_x_entry[i] - Cx)**2 + (p_y_entry[i] - Cy)**2
        d0 = pyo.sqrt(d0_sq + 1e-6)
        d0_prime_sq = (p_x_entry[i] - Cx_prime)**2 + (p_y_entry[i] - Cy_prime)**2
        
        # Tangent Leg (d_L)
        d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
        
        # Angles
        term1 = r / (d0 + 1e-6)
        term2 = (r**2 + d0_sq - d0_prime_sq) / (2 * r * (d0 + 1e-6))
        theta1 = pyo.acos(term1)
        theta2 = pyo.acos(term2)
        
        if p_long_arc[i]:
            theta_rad = 2*3.14159 - (theta1 + theta2)
        else:
            theta_rad = theta2 - theta1
            
        # Distances
        dist_turn = r * theta_rad
        dist_final = m.d[i]
        
        # Time Calculation using FIXED SPEEDS
        # (nm / knots) * 3600 = seconds
        t_L = (d_L / v_L_const) * 3600
        t_turn = (dist_turn / v_theta_const) * 3600
        t_final = (dist_final / v_f_const) * 3600
        
        return p_tau[i] + t_L + t_turn + t_final

    # --- D. Constraints ---
    
    # 1. Physics Constraint
    def physics_rule(m, i):
        return m.t[i] == calc_travel_time_expr(m, i)
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # 2. Robust Separation Constraint
    def separation_rule(m, i):
        prev_i = i - 1
        if prev_i not in m.I:
            return pyo.Constraint.Skip
        
        # Constraint: Time[i] >= Time[i-1] + 64 - Slack[i]
        return m.t[i] >= m.t[prev_i] + T_SEP_SEC - m.slack[i]
    m.c_separation = pyo.Constraint(m.I, rule=separation_rule)
    
    # --- E. Objective ---
    # Minimize Last Arrival Time + Penalty for Violations
    # Weight = 1000. This means the solver will do EVERYTHING possible to extend d_i
    # before it dares to use 1 second of slack.
    last_id = flight_ids[-1]
    m.obj = pyo.Objective(expr=m.t[last_id] + 1000 * sum(m.slack[i] for i in m.I), sense=pyo.minimize)
    
    # --- F. Solve ---
    solver = SolverFactory('ipopt')
    if not debug: solver.options['print_level'] = 0
    
    # Ipopt options for robustness
    solver.options['max_iter'] = 10000
    solver.options['tol'] = 1e-6
    
    solver.solve(m, tee=debug)
    
    # --- G. Extract Results ---
    results = []
    for i in m.I:
        results.append({
            'aircraft_id': i,
            'entry_time': p_tau[i],
            'optimized_d_i': pyo.value(m.d[i]),
            'arrival_time_faf': pyo.value(m.t[i]),
            'scheduled_delay': pyo.value(m.t[i]) - p_tau[i]
        })
        
    return pd.DataFrame(results)

# --- Run Optimization ---
print("Running Robust Optimization (Fixed Speed, d_i only)...")
df_results = solve_schedule_optimization_fixed_speed_robust(df_arrivals, debug=True)

# Merge and Display
df_final = pd.merge(df_arrivals, df_results[['aircraft_id', 'optimized_d_i', 'arrival_time_faf']], on='aircraft_id')

# Calculate achieved separation
df_final['prev_arrival'] = df_final['arrival_time_faf'].shift(1)
df_final['actual_separation'] = df_final['arrival_time_faf'] - df_final['prev_arrival']

# Check total violation
if df_final['actual_separation'].any() < T_SEP_SEC:
    print(f"WARNING: Traffic demand exceeded capacity. Some aircraft were not separated by {T_SEP_SEC}s.")
else:
    print(f"SUCCESS: All aircraft separated by {T_SEP_SEC}s using only path extension.")

df_final

Running Robust Optimization (Fixed Speed, d_i only)...


Ipopt 3.14.19: max_iter=10000
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       80
Number of nonzeros in inequality constraint Jacobian.:      117
Number of nonzeros in Lagrangian Hessian.............:       40

Total number of variables............................:      120
                     variables with only lower bounds:       80
                variables with lower and upper bounds:       40
                     variables with only upper bounds:        0
Total number of equality constraints...............

,entry_time,corner,enter_fix_name,x_entry,y_entry,x_faf,y_faf,r_turn,is_north,long_arc,aircraft_id,optimized_d_i,arrival_time_faf,prev_arrival,actual_separation
0,178.914341,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,1,6.770463e+00,754.671534,NaN,NaN
1,331.102014,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,2,1.196538e+01,1042.255399,754.671534,287.583865
2,431.010455,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,3,1.174637e+00,1353.291614,1042.255399,311.036216
3,665.574532,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,4,1.463734e+01,1460.402431,1353.291614,107.110817
4,710.879026,NorthEast,LOGEN,19.475811,21.254227,-5.090967,0.001485,6.0,True,1,5,1.038317e+00,1573.105783,1460.402431,112.703352
5,764.391897,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,6,5.600574e-01,1661.537721,1573.105783,88.431939
6,844.508337,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,7,1.934483e+01,1738.956361,1661.537721,77.418640
7,1061.964834,NorthEast,LOGEN,19.475811,21.254227,-5.090967,0.001485,6.0,True,1,8,1.174623e-01,1887.025048,1738.956361,148.068686
8,1098.527750,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,9,1.739304e+01,1957.519027,1887.025048,70.493979
9,1157.370978,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,10,1.886108e+01,2028.426240,1957.519027,70.907213


# Setup II: Co-optimize $d_i$ and segments speed 

In [5]:
# ==========================================
# 7. CO-OPTIMIZATION MODEL (Hard Constraints)
#    Variables: d_i, v_L, v_theta, v_f
#    Constraint: STRICT 64s separation
# ==========================================

def solve_co_optimization_no_slack(df_arrivals, debug=False):
    """
    Implements Section 2.3: Co-optimize d_i and Speeds.
    NO Slack variables. Enforces strict T_sep.
    May result in 'Infeasible' if traffic demand is too high.
    """
    
    m = pyo.ConcreteModel()
    
    # --- A. Sets & Parameters ---
    flight_ids = df_arrivals['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Dictionaries
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x_entry = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y_entry = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_is_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long_arc = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    # Geometry Constants
    r = R_TURN_NM
    x_faf = processed_fixes['VINII']['x']
    y_faf = processed_fixes['VINII']['y']
    
    # --- B. Decision Variables ---
    
    # 1. Path Extension (0 to 20 nm)
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    
    # 2. Arrival Time (seconds)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=lambda m, i: p_tau[i] + 600)
    
    # 3. Variable Speeds (Knots) - With Bounds from your sets
    # We initialize to higher speeds to encourage "fastest possible" flow first
    m.v_L = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(180, 250), initialize=210)
    m.v_theta = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=160)
    m.v_f = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=140)

    # --- C. Geometry Expressions ---
    def calc_distances_expr(m, i):
        y_center_offset = r if p_is_north[i] else -r
        Cx = x_faf - m.d[i]
        Cy = y_faf + y_center_offset
        Cx_prime = x_faf - m.d[i]
        Cy_prime = y_faf
        
        d0_sq = (p_x_entry[i] - Cx)**2 + (p_y_entry[i] - Cy)**2
        d0 = pyo.sqrt(d0_sq + 1e-6)
        d0_prime_sq = (p_x_entry[i] - Cx_prime)**2 + (p_y_entry[i] - Cy_prime)**2
        
        d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
        
        term1 = r / (d0 + 1e-6)
        term2 = (r**2 + d0_sq - d0_prime_sq) / (2 * r * (d0 + 1e-6))
        theta1 = pyo.acos(term1)
        theta2 = pyo.acos(term2)
        
        if p_long_arc[i]:
            theta_rad = 2*3.14159 - (theta1 + theta2)
        else:
            theta_rad = theta2 - theta1
            
        d_theta = r * theta_rad
        d_final = m.d[i]
        
        return d_L, d_theta, d_final

    # --- D. Constraints ---
    
    # 1. Physics: Time depends on Distance and Variable Speeds
    def physics_rule(m, i):
        d_L, d_theta, d_final = calc_distances_expr(m, i)
        t_L = (d_L / m.v_L[i]) * 3600
        t_turn = (d_theta / m.v_theta[i]) * 3600
        t_final = (d_final / m.v_f[i]) * 3600
        return m.t[i] == p_tau[i] + t_L + t_turn + t_final
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # 2. HARD Separation Constraint (Strict Inequality)
    def separation_rule(m, i):
        prev_i = i - 1
        if prev_i not in m.I:
            return pyo.Constraint.Skip
        # STRICT: Current time must be >= Previous time + 64s
        return m.t[i] >= m.t[prev_i] + T_SEP_SEC
    m.c_separation = pyo.Constraint(m.I, rule=separation_rule)
    
    # 3. Speed Monotonicity Constraints (v_L >= v_theta >= v_f)
    def speed_mono_1(m, i):
        return m.v_L[i] >= m.v_theta[i]
    m.c_speed_1 = pyo.Constraint(m.I, rule=speed_mono_1)

    def speed_mono_2(m, i):
        return m.v_theta[i] >= m.v_f[i]
    m.c_speed_2 = pyo.Constraint(m.I, rule=speed_mono_2)
    
    # --- E. Objective ---
    # Minimize Makespan (Arrival time of the last aircraft)
    last_id = flight_ids[-1]
    m.obj = pyo.Objective(expr=m.t[last_id], sense=pyo.minimize)
    
    # --- F. Solve ---
    solver = SolverFactory('ipopt')
    # Increase iter limit because finding a feasible point with hard constraints is harder
    solver.options['max_iter'] = 10000
    
    if not debug:
        solver.options['print_level'] = 0
        
    results_obj = solver.solve(m, tee=debug)
    
    # Check if optimal
    status = results_obj.solver.termination_condition
    print(f"Solver Status: {status}")

    # --- G. Extract ---
    results = []
    # Even if infeasible, we try to extract values to see where it failed
    try:
        for i in m.I:
            results.append({
                'aircraft_id': i,
                'entry_time': p_tau[i],
                'optimized_d_i': pyo.value(m.d[i]),
                'arrival_time_faf': pyo.value(m.t[i]),
                'v_L': pyo.value(m.v_L[i]),
                'v_theta': pyo.value(m.v_theta[i]),
                'v_f': pyo.value(m.v_f[i])
            })
    except ValueError:
        print("Could not extract values (Solver likely failed completely).")
        return pd.DataFrame()
        
    return pd.DataFrame(results)

# --- Run ---
print("Running Co-Optimization (Hard Constraints)...")
df_hard_results = solve_co_optimization_no_slack(df_arrivals, debug=True)

if not df_hard_results.empty:
    df_final_hard = pd.merge(df_arrivals, df_hard_results, on='aircraft_id')
    # Calculate achieved separation
    df_final_hard['prev_arrival'] = df_final_hard['arrival_time_faf'].shift(1)
    df_final_hard['separation'] = df_final_hard['arrival_time_faf'] - df_final_hard['prev_arrival']
    
    display(df_final_hard[['aircraft_id', 'optimized_d_i', 'v_L', 'v_f', 'arrival_time_faf', 'separation']])
else:
    print("Optimization failed to find a feasible solution.")

Running Co-Optimization (Hard Constraints)...


Ipopt 3.14.19: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      200
Number of nonzeros in inequality constraint Jacobian.:      238
Number of nonzeros in Lagrangian Hessian.............:      280

Total number of variables............................:      200
                     variables with only lower bounds:       40
                variables with lower and upper bounds:      160
                     variables with only upper bounds:        0
Total number of equality constraints.................:       

,aircraft_id,optimized_d_i,v_L,v_f,arrival_time_faf,separation
0,1,6.633196e+00,227.241804,148.933212,737.869074,NaN
1,2,1.135959e+01,225.454169,147.310341,1022.211971,284.342897
2,3,1.575888e+00,235.317469,150.927837,1304.628748,282.416777
3,4,1.328034e+01,224.874922,146.441759,1420.005333,115.376586
4,5,1.654260e+00,234.644886,150.794521,1539.543011,119.537678
5,6,1.528395e+00,235.512352,150.983376,1635.362036,95.819025
6,7,1.622913e+01,219.396196,140.733843,1721.551769,86.189734
7,8,4.524368e-01,242.632539,153.060672,1814.083476,92.531707
8,9,1.434919e+01,224.331532,145.719531,1890.567625,76.484149
9,10,1.481063e+01,224.223109,145.431735,1967.369771,76.802146


### With Slack Variables

In [6]:
def solve_schedule_optimization_robust(df_arrivals, debug=False):
    m = pyo.ConcreteModel()
    
    # Sets & Parameters (Same as before)
    flight_ids = df_arrivals['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x_entry = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y_entry = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_is_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long_arc = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    r = R_TURN_NM
    x_faf = processed_fixes['VINII']['x']
    y_faf = processed_fixes['VINII']['y']
    
    # --- Decision Variables ---
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=lambda m, i: p_tau[i] + 600)
    m.v_L = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(180, 250), initialize=210)
    m.v_theta = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=160)
    m.v_f = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=140)
    
    # NEW: Slack Variable (Amount of separation violation allowed)
    # We initialize it to 0.0. If the solver uses this, it admits defeat on separation.
    m.slack = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=0.0)

    # --- Expressions ---
    def calc_distances_expr(m, i):
        y_center_offset = r if p_is_north[i] else -r
        Cx = x_faf - m.d[i]
        Cy = y_faf + y_center_offset
        Cx_prime = x_faf - m.d[i]
        Cy_prime = y_faf
        
        d0_sq = (p_x_entry[i] - Cx)**2 + (p_y_entry[i] - Cy)**2
        d0 = pyo.sqrt(d0_sq + 1e-6) # Safety epsilon
        d0_prime_sq = (p_x_entry[i] - Cx_prime)**2 + (p_y_entry[i] - Cy_prime)**2
        
        # Tangent d_L
        d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
        
        # Angles
        term1 = r / (d0 + 1e-6)
        term2 = (r**2 + d0_sq - d0_prime_sq) / (2 * r * (d0 + 1e-6))
        
        theta1 = pyo.acos(term1)
        theta2 = pyo.acos(term2)
        
        if p_long_arc[i]:
            theta_rad = 2*3.14159 - (theta1 + theta2)
        else:
            theta_rad = theta2 - theta1
            
        d_theta = r * theta_rad
        d_final = m.d[i]
        
        return d_L, d_theta, d_final

    # --- Constraints ---
    def physics_rule(m, i):
        d_L, d_theta, d_final = calc_distances_expr(m, i)
        t_L = (d_L / m.v_L[i]) * 3600
        t_turn = (d_theta / m.v_theta[i]) * 3600
        t_final = (d_final / m.v_f[i]) * 3600
        return m.t[i] == p_tau[i] + t_L + t_turn + t_final
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # NEW: Robust Separation Rule
    def separation_rule(m, i):
        prev_i = i - 1
        if prev_i not in m.I:
            return pyo.Constraint.Skip
        
        # Constraint: t[i] must be at least t[i-1] + 64, 
        # BUT we subtract slack[i]. If slack[i] is positive, we allow a violation.
        return m.t[i] >= m.t[prev_i] + T_SEP_SEC - m.slack[i]
        
    m.c_separation = pyo.Constraint(m.I, rule=separation_rule)
    
    # Speed constraints (same as before)
    m.c_speed_1 = pyo.Constraint(m.I, rule=lambda m, i: m.v_L[i] >= m.v_theta[i])
    m.c_speed_2 = pyo.Constraint(m.I, rule=lambda m, i: m.v_theta[i] >= m.v_f[i])
    
    # --- NEW Objective ---
    # Minimize Makespan + Huge Penalty for violating separation
    # Penalty weight = 1000 (1 second of violation costs as much as 1000 seconds of flight time)
    last_id = flight_ids[-1]
    m.obj = pyo.Objective(expr=m.t[last_id] + 1000 * sum(m.slack[i] for i in m.I), sense=pyo.minimize)
    
    # --- Solve ---
    solver = SolverFactory('ipopt')
    if not debug: solver.options['print_level'] = 0
    
    # Increase max iterations to handle the larger search space
    solver.options['max_iter'] = 10000 
    solver.solve(m, tee=debug)
    
    # --- Extract ---
    results = []
    for i in m.I:
        results.append({
            'aircraft_id': i,
            'd_i': pyo.value(m.d[i]),
            'arrival_time': pyo.value(m.t[i]),
            'v_L': pyo.value(m.v_L[i]),
            'v_theta': pyo.value(m.v_theta[i]),
            'v_f': pyo.value(m.v_f[i])
        })
    return pd.DataFrame(results)

print("Optimizing Schedule...")
df_opt_results = solve_schedule_optimization_robust(df_arrivals, debug=True)
df_final = pd.merge(df_arrivals, df_opt_results, on='aircraft_id')

# Calculate achieved separation
df_final['prev_arrival'] = df_final['arrival_time'].shift(1)
df_final['actual_separation'] = df_final['arrival_time'] - df_final['prev_arrival']

df_final

Optimizing Schedule...


Ipopt 3.14.19: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      200
Number of nonzeros in inequality constraint Jacobian.:      277
Number of nonzeros in Lagrangian Hessian.............:      280

Total number of variables............................:      240
                     variables with only lower bounds:       80
                variables with lower and upper bounds:      160
                     variables with only upper bounds:        0
Total number of equality constraints.................:       

,entry_time,corner,enter_fix_name,x_entry,y_entry,x_faf,y_faf,r_turn,is_north,long_arc,aircraft_id,d_i,arrival_time,v_L,v_theta,v_f,prev_arrival,actual_separation
0,178.914341,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,1,6.633196e+00,737.869074,227.241804,176.168813,148.933212,NaN,NaN
1,331.102014,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,2,1.135959e+01,1022.211971,225.454169,174.299456,147.310341,737.869074,284.342897
2,431.010455,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,3,1.575888e+00,1304.628748,235.317469,183.061713,150.927837,1022.211971,282.416777
3,665.574532,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,4,1.328034e+01,1420.005333,224.874922,173.394052,146.441759,1304.628748,115.376586
4,710.879026,NorthEast,LOGEN,19.475811,21.254227,-5.090967,0.001485,6.0,True,1,5,1.654260e+00,1539.543011,234.644886,182.540300,150.794521,1420.005333,119.537678
5,764.391897,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,6,1.528395e+00,1635.362036,235.512352,183.261317,150.983376,1539.543011,95.819025
6,844.508337,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,7,1.622913e+01,1721.551769,219.396196,165.927949,140.733843,1635.362036,86.189734
7,1061.964834,NorthEast,LOGEN,19.475811,21.254227,-5.090967,0.001485,6.0,True,1,8,4.524368e-01,1814.083476,242.632539,191.458616,153.060672,1721.551769,92.531707
8,1098.527750,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,9,1.434919e+01,1890.567625,224.331532,172.667172,145.719531,1814.083476,76.484149
9,1157.370978,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,10,1.481063e+01,1967.369771,224.223109,172.306859,145.431735,1890.567625,76.802146


In [7]:
# Calculate Delay (Arrival Time - Minimum Possible Time)
# Note: This is simplified; true delay is typically (Actual - Scheduled)
# Here we just look at the timeline.
df_final['transit_time'] = df_final['arrival_time'] - df_final['entry_time']

print("\nOptimization Complete.")
display(df_final[['aircraft_id', 'corner', 'entry_time', 'd_i', 'arrival_time', 'transit_time']])


Optimization Complete.


,aircraft_id,corner,entry_time,d_i,arrival_time,transit_time
0,1,NorthWest,178.914341,6.633196e+00,737.869074,558.954733
1,2,NorthWest,331.102014,1.135959e+01,1022.211971,691.109956
2,3,SouthEast,431.010455,1.575888e+00,1304.628748,873.618292
3,4,NorthWest,665.574532,1.328034e+01,1420.005333,754.430801
4,5,NorthEast,710.879026,1.654260e+00,1539.543011,828.663985
5,6,SouthEast,764.391897,1.528395e+00,1635.362036,870.970139
6,7,SouthWest,844.508337,1.622913e+01,1721.551769,877.043433
7,8,NorthEast,1061.964834,4.524368e-01,1814.083476,752.118642
8,9,SouthWest,1098.527750,1.434919e+01,1890.567625,792.039875
9,10,NorthWest,1157.370978,1.481063e+01,1967.369771,809.998792


# Two Stage Optimization
 - To enforce the segment speeds only from speed option sets: 

$\mathcal V_L=\{180,190,200,210,220,230,240,250\} \\
\mathcal V_{\theta}=\{130,140,150,160,170,180,190,200\} \\
\mathcal V_F=\{130,140,150,160,170,180,190,200\} \\$

In [8]:
def solve_two_stage_discrete(df_arrivals, debug=False):
    print("--- STAGE 1: Solving Continuous Relaxation ---")
    # 1. Solve with continuous speeds (using the function we wrote previously)
    # Note: Using the robust version (with slack) ensures we get a solution to round
    df_stage1 = solve_schedule_optimization_robust(df_arrivals, debug=False)
    
    if df_stage1.empty:
        print("Stage 1 failed. Cannot proceed.")
        return pd.DataFrame()
        
    print("--- Rounding Speeds ---")
    # 2. Round speeds to nearest 10
    def round_to_10(x): return 10 * round(x / 10)
    
    # Create a dictionary of FIXED speeds for Stage 2
    fixed_v_L = {row['aircraft_id']: round_to_10(row['v_L']) for _, row in df_stage1.iterrows()}
    fixed_v_theta = {row['aircraft_id']: round_to_10(row['v_theta']) for _, row in df_stage1.iterrows()}
    fixed_v_f = {row['aircraft_id']: round_to_10(row['v_f']) for _, row in df_stage1.iterrows()}
    
    # Note: You can do logic here. E.g., "If rounding up violates speed limit, round down"
    # For now, we assume simple rounding keeps us within [180, 250] approx.
    
    print("--- STAGE 2: Re-optimizing d_i with Fixed Discrete Speeds ---")
    
    # 3. Build Stage 2 Model (Similar to "Fixed Speed" model, but speeds vary per aircraft)
    m = pyo.ConcreteModel()
    m.I = pyo.Set(initialize=df_arrivals['aircraft_id'].tolist())
    
    # Parameters
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    # GEOMETRY CONSTANTS
    r = R_TURN_NM
    x_faf, y_faf = processed_fixes['VINII']['x'], processed_fixes['VINII']['y']
    
    # VARIABLES
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals)
    m.slack = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=0.0) # Keep slack for safety

    # EXPRESSIONS & CONSTRAINTS
    def physics_rule(m, i):
        # --- Geometry Calc (Same as before) ---
        y_off = r if p_north[i] else -r
        Cx, Cy = x_faf - m.d[i], y_faf + y_off
        d0 = pyo.sqrt((p_x[i]-Cx)**2 + (p_y[i]-Cy)**2 + 1e-6)
        d0_p = pyo.sqrt((p_x[i]-(x_faf-m.d[i]))**2 + (p_y[i]-y_faf)**2)
        d_L = pyo.sqrt(d0**2 - r**2 + 1e-6)
        
        theta1 = pyo.acos(r/(d0+1e-6))
        theta2 = pyo.acos((r**2 + d0**2 - d0_p**2)/(2*r*(d0+1e-6)))
        theta = (2*3.14159 - (theta1+theta2)) if p_long[i] else (theta2-theta1)
        
        # --- TIME CALC USING FIXED ROUNDED SPEEDS ---
        # We use the fixed dictionary here!
        v_L_val = fixed_v_L[i]  
        v_theta_val = fixed_v_theta[i]
        v_f_val = fixed_v_f[i]
        
        # If you optimized all 3, round all 3 and put them here
        
        return m.t[i] == p_tau[i] + (d_L/v_L_val)*3600 + ((r*theta)/v_theta_val)*3600 + (m.d[i]/v_f_val)*3600

    m.phys = pyo.Constraint(m.I, rule=physics_rule)
    
    def sep_rule(m, i):
        prev = i-1
        if prev not in m.I: return pyo.Constraint.Skip
        return m.t[i] >= m.t[prev] + T_SEP_SEC - m.slack[i]
    m.sep = pyo.Constraint(m.I, rule=sep_rule)
    
    # Objective: Minimize Makespan + Penalty
    last = m.I.last()
    m.obj = pyo.Objective(expr=m.t[last] + 1000*sum(m.slack[i] for i in m.I), sense=pyo.minimize)
    
    # SOLVE
    solver = SolverFactory('ipopt')
    solver.options['print_level'] = 0
    solver.solve(m)
    
    # Extract
    res = []
    for i in m.I:
        res.append({
            'aircraft_id': i,
            'optimized_d_i': pyo.value(m.d[i]),
            'v_L_discrete': fixed_v_L[i], # This is now an integer multiple of 10
            'v_theta_discrete': fixed_v_theta[i],
            'v_f_discrete': fixed_v_f[i],
            'arrival_time': pyo.value(m.t[i])
        })
    return pd.DataFrame(res)

# Run it
df_discrete = solve_two_stage_discrete(df_arrivals)

# Calculate achieved separation
df_discrete = pd.merge(df_arrivals, df_discrete, on='aircraft_id')
df_discrete['prev_arrival'] = df_discrete['arrival_time'].shift(1)
df_discrete['actual_separation'] = df_discrete['arrival_time'] - df_discrete['prev_arrival']

display(df_discrete)

--- STAGE 1: Solving Continuous Relaxation ---
--- Rounding Speeds ---
--- STAGE 2: Re-optimizing d_i with Fixed Discrete Speeds ---


,entry_time,corner,enter_fix_name,x_entry,y_entry,x_faf,y_faf,r_turn,is_north,long_arc,aircraft_id,optimized_d_i,v_L_discrete,v_theta_discrete,v_f_discrete,arrival_time,prev_arrival,actual_separation
0,178.914341,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,1,6.706462e+00,230,180,150,732.052388,NaN,NaN
1,331.102014,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,2,1.136387e+01,230,170,150,1019.072943,732.052388,287.020555
2,431.010455,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,3,1.653748e+00,240,180,150,1304.255633,1019.072943,285.182690
3,665.574532,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,4,1.329309e+01,220,170,150,1421.625891,1304.255633,117.370258
4,710.879026,NorthEast,LOGEN,19.475811,21.254227,-5.090967,0.001485,6.0,True,1,5,1.397990e+00,230,180,150,1543.645146,1421.625891,122.019255
5,764.391897,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,6,1.660103e+00,240,180,150,1637.884206,1543.645146,94.239060
6,844.508337,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,7,1.663586e+01,220,170,140,1723.844342,1637.884206,85.960137
7,1061.964834,NorthEast,LOGEN,19.475811,21.254227,-5.090967,0.001485,6.0,True,1,8,3.370127e-01,240,190,150,1816.960452,1723.844342,93.116109
8,1098.527750,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,9,1.446496e+01,220,170,150,1891.782420,1816.960452,74.821969
9,1157.370978,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,10,1.502988e+01,220,170,150,1966.843458,1891.782420,75.061038


## MINLP formulation for intervaled segment speed
 - Introduce the binary variables to select exactly one speed for each segment

In [9]:
# ==========================================
# 8. ROBUST MINLP EXACT FORMULATION
#    Discrete Speeds + Slack Variables
# ==========================================

def solve_exact_discrete_minlp_robust(df_arrivals, debug=False):
    m = pyo.ConcreteModel()
    
    # --- A. Sets & Parameters ---
    flight_ids = df_arrivals['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Parameters
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    r = R_TURN_NM
    x_faf, y_faf = processed_fixes['VINII']['x'], processed_fixes['VINII']['y']
    
    # Discrete Speed Sets
    Set_VL = [180.0, 190.0, 200.0, 210.0, 220.0, 230.0, 240.0, 250.0]
    Set_Vtheta = [130.0, 140.0, 150.0, 160.0, 170.0, 180.0, 190.0, 200.0]
    Set_VF = [130.0, 140.0, 150.0, 160.0, 170.0, 180.0, 190.0, 200.0]
    
    m.Set_VL = pyo.Set(initialize=Set_VL)
    m.Set_Vtheta = pyo.Set(initialize=Set_Vtheta)
    m.Set_VF = pyo.Set(initialize=Set_VF)

    # --- B. Decision Variables ---
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=lambda m, i: p_tau[i] + 600)
    
    # CRITICAL: Slack Variable to prevent "Cycling/Infeasible" errors
    m.slack = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=0.0)

    # Binary Speed Selection
    m.z_L = pyo.Var(m.I, m.Set_VL, domain=pyo.Binary)
    m.z_theta = pyo.Var(m.I, m.Set_Vtheta, domain=pyo.Binary)
    m.z_F = pyo.Var(m.I, m.Set_VF, domain=pyo.Binary)

    # --- C. Constraints ---

    # 1. Select Exactly One
    m.c_sel_L = pyo.Constraint(m.I, rule=lambda m, i: sum(m.z_L[i, v] for v in m.Set_VL) == 1)
    m.c_sel_theta = pyo.Constraint(m.I, rule=lambda m, i: sum(m.z_theta[i, v] for v in m.Set_Vtheta) == 1)
    m.c_sel_F = pyo.Constraint(m.I, rule=lambda m, i: sum(m.z_F[i, v] for v in m.Set_VF) == 1)

    # 2. Effective Speed Expressions
    def get_vL(m, i): return sum(v * m.z_L[i, v] for v in m.Set_VL)
    def get_vTheta(m, i): return sum(v * m.z_theta[i, v] for v in m.Set_Vtheta)
    def get_vF(m, i): return sum(v * m.z_F[i, v] for v in m.Set_VF)

    # 3. Monotonicity
    m.c_mono_1 = pyo.Constraint(m.I, rule=lambda m, i: get_vL(m, i) >= get_vTheta(m, i))
    m.c_mono_2 = pyo.Constraint(m.I, rule=lambda m, i: get_vTheta(m, i) >= get_vF(m, i))

    # 4. Physics (Safe Geometry + Discrete Speed)
    def physics_rule(m, i):
        y_off = r if p_north[i] else -r
        Cx, Cy = x_faf - m.d[i], y_faf + y_off
        
        # Safe Geometry Logic
        d0_sq = (p_x[i]-Cx)**2 + (p_y[i]-Cy)**2
        d0_p_sq = (p_x[i]-(x_faf-m.d[i]))**2 + (p_y[i]-y_faf)**2
        
        safe_term = pyo.Expr_if(d0_sq - r**2 > 1e-6, d0_sq - r**2, 0.0)
        d_L = pyo.sqrt(safe_term + 1e-6)
        
        # d0 calculation for angles
        d0 = pyo.sqrt(d0_sq + 1e-6)
        
        term1 = r / d0
        term2 = (r**2 + d0_sq - d0_p_sq) / (2*r*d0)
        
        theta1 = pyo.acos(term1)
        theta2 = pyo.acos(term2)
        theta_rad = (2*3.14159 - (theta1+theta2)) if p_long[i] else (theta2-theta1)
        
        d_theta = r * theta_rad
        d_final = m.d[i]
        
        # Time = Distance / DiscreteSpeed
        t_L = (d_L / get_vL(m, i)) * 3600
        t_turn = (d_theta / get_vTheta(m, i)) * 3600
        t_final = (d_final / get_vF(m, i)) * 3600
        
        return m.t[i] == p_tau[i] + t_L + t_turn + t_final

    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)

    # 5. Separation with Slack (CRITICAL FIX)
    def separation_rule(m, i):
        prev_i = i - 1
        if prev_i not in m.I: return pyo.Constraint.Skip
        # The slack allows the NLP subproblem to converge even if MILP guess is slightly off
        return m.t[i] >= m.t[prev_i] + T_SEP_SEC - m.slack[i]
    m.c_separation = pyo.Constraint(m.I, rule=separation_rule)

    # --- D. Objective ---
    # Penalty of 5000 ensures slack is only used if absolutely necessary
    last_id = flight_ids[-1]
    m.obj = pyo.Objective(expr=m.t[last_id] + 5000 * sum(m.slack[i] for i in m.I), sense=pyo.minimize)

    # --- E. Solve ---
    solver = SolverFactory('mindtpy')
    
    print("Running MindtPy (Robust)...")
    try:
        # Increase nlp_iteration_limit to give Ipopt more time
        results = solver.solve(m, mip_solver='glpk', nlp_solver='ipopt', 
                               tee=debug, 
                               iteration_limit=30, # Max outer iterations
                               nlp_iteration_limit=3000) # Max Ipopt iterations
        
    except Exception as e:
        print(f"Solver Crashed: {e}")
        return pd.DataFrame()

    # --- F. Safety Check Before Extraction ---
    # This prevents the "No value for uninitialized VarData" error
    if results.solver.termination_condition == TerminationCondition.infeasible:
        print("MindtPy declared problem INFEASIBLE.")
        return pd.DataFrame()
        
    # Check if a Primal solution actually exists
    # MindtPy sometimes returns 'optimal' status even if it only found a bound
    try:
        pyo.value(m.d[flight_ids[0]])
    except ValueError:
        print("MindtPy failed to load a valid Primal solution (likely cycling).")
        return pd.DataFrame()

    # --- G. Extract ---
    res = []
    for i in m.I:
        # Extract selected speeds
        vL_val = sum(v for v in m.Set_VL if pyo.value(m.z_L[i, v]) > 0.5)
        vTheta_val = sum(v for v in m.Set_Vtheta if pyo.value(m.z_theta[i, v]) > 0.5)
        vF_val = sum(v for v in m.Set_VF if pyo.value(m.z_F[i, v]) > 0.5)
        
        res.append({
            'aircraft_id': i,
            'd_i': pyo.value(m.d[i]),
            'v_L': vL_val,
            'v_theta': vTheta_val,
            'v_f': vF_val,
            'arrival_time': pyo.value(m.t[i]),
            'slack': pyo.value(m.slack[i])
        })
        
    return pd.DataFrame(res)

# --- Instructions to Run ---
# Uncomment the line below ONLY if you have 'glpk' installed alongside 'ipopt'
df_minlp = solve_exact_discrete_minlp(df_arrivals, debug=True)

# Calculate achieved separation
df_minlp = pd.merge(df_arrivals, df_minlp, on='aircraft_id')
df_minlp['prev_arrival'] = df_minlp['arrival_time'].shift(1)
df_minlp['actual_separation'] = df_minlp['arrival_time'] - df_minlp['prev_arrival']

display(df_minlp)

NameError: name 'solve_exact_discrete_minlp' is not defined

# Setup III: Co-optimize $d_i$, segments speed, and the arrival sequence (Do Not Follow FCFS Rule)
 - This is the most computationally complex formulation: Mixed-Integer Nonlinear Programming (MINLP).

In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd
import itertools

# ==========================================
# 9. MINLP SEQUENCING + TRAJECTORY
#    Variables: d_i, Speeds (Continuous), Sequence (Binary)
# ==========================================

def solve_sequencing_minlp(df_arrivals, debug=False):
    """
    Implements Setup III: Co-optimize Sequence, Speed, and d_i.
    Uses Big-M formulation for disjunctive constraints.
    """
    
    # --- Limit Problem Size for Demo ---
    # MINLP scales factorially. We limit to first 5 aircraft for this test.
    df_subset = df_arrivals.copy() 
    print(f"Solving Optimal Sequence for {len(df_subset)} aircraft...")
    
    m = pyo.ConcreteModel()
    
    # --- A. Sets & Parameters ---
    flight_ids = df_subset['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Create Set of Pairs (i, j) where i < j (to avoid double counting)
    # We will decide sequence for every unique pair
    m.Pairs = pyo.Set(initialize=[(i, j) for i in flight_ids for j in flight_ids if i < j])
    
    p_tau = df_subset.set_index('aircraft_id')['entry_time'].to_dict()
    p_x = df_subset.set_index('aircraft_id')['x_entry'].to_dict()
    p_y = df_subset.set_index('aircraft_id')['y_entry'].to_dict()
    p_north = df_subset.set_index('aircraft_id')['is_north'].to_dict()
    p_long = df_subset.set_index('aircraft_id')['long_arc'].to_dict()
    
    r = R_TURN_NM
    x_faf, y_faf = processed_fixes['VINII']['x'], processed_fixes['VINII']['y']
    
    # Big-M Parameter (Must be large enough to make constraint inactive when y=0)
    # But not too large to cause numerical errors. 
    # Max logical separation ~ 1 hour (3600s)
    M = 5000.0 

    # --- B. Decision Variables ---
    
    # Trajectory Variables
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0, 10000), initialize=lambda m, i: p_tau[i] + 600)
    
    # Continuous Speed Variables (Section 2.3 logic)
    m.v_L = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(180, 250), initialize=210)
    m.v_theta = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=160)
    m.v_f = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=140)
    
    # SEQUENCE VARIABLE (Binary)
    # y[i,j] = 1 if i arrives BEFORE j
    # y[i,j] = 0 if j arrives BEFORE i
    m.y = pyo.Var(m.Pairs, domain=pyo.Binary)
    
    # Makespan Variable (The scalar we want to minimize)
    m.makespan = pyo.Var(domain=pyo.NonNegativeReals)

    # --- C. Trajectory Physics Constraints (Same as before) ---
    def calc_distances_expr(m, i):
        y_off = r if p_north[i] else -r
        Cx, Cy = x_faf - m.d[i], y_faf + y_off
        d0 = pyo.sqrt((p_x[i]-Cx)**2 + (p_y[i]-Cy)**2 + 1e-6)
        d0_p = pyo.sqrt((p_x[i]-(x_faf-m.d[i]))**2 + (p_y[i]-y_faf)**2)
        d_L = pyo.sqrt(d0**2 - r**2 + 1e-6)
        theta1 = pyo.acos(r/(d0+1e-6))
        theta2 = pyo.acos((r**2 + d0**2 - d0_p**2)/(2*r*(d0+1e-6)))
        theta_rad = (2*3.14159 - (theta1+theta2)) if p_long[i] else (theta2-theta1)
        return d_L, r * theta_rad, m.d[i]

    def physics_rule(m, i):
        d_L, d_theta, d_final = calc_distances_expr(m, i)
        t_L = (d_L / m.v_L[i]) * 3600
        t_turn = (d_theta / m.v_theta[i]) * 3600
        t_final = (d_final / m.v_f[i]) * 3600
        # Constraint: Arrival Time must equal Entry + Travel
        # Note: We use inequality (>=) here to allow "Delay" at the entry point 
        # if optimization requires holding, though usually equality is preferred. 
        # For this rigorous formulation, we stick to equality (physics).
        return m.t[i] == p_tau[i] + t_L + t_turn + t_final
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # Speed Monotonicity
    m.c_speed1 = pyo.Constraint(m.I, rule=lambda m, i: m.v_L[i] >= m.v_theta[i])
    m.c_speed2 = pyo.Constraint(m.I, rule=lambda m, i: m.v_theta[i] >= m.v_f[i])

    # --- D. Big-M Sequencing Constraints ---
    
    def seq_rule_1(m, i, j):
        # Constraint: If y[i,j]=1 (i before j), then t[j] >= t[i] + Sep
        # If y[i,j]=0, the -M term makes this constraint trivial (-5000), so it's ignored
        return m.t[j] >= m.t[i] + T_SEP_SEC - M * (1 - m.y[i, j])
    m.c_seq_1 = pyo.Constraint(m.Pairs, rule=seq_rule_1)
    
    def seq_rule_2(m, i, j):
        # Constraint: If y[i,j]=0 (j before i), then t[i] >= t[j] + Sep
        return m.t[i] >= m.t[j] + T_SEP_SEC - M * (m.y[i, j])
    m.c_seq_2 = pyo.Constraint(m.Pairs, rule=seq_rule_2)

    # --- E. Objective: Minimize Makespan ---
    
    # Constrain makespan variable to be >= all arrival times
    def makespan_def(m, i):
        return m.makespan >= m.t[i]
    m.c_makespan = pyo.Constraint(m.I, rule=makespan_def)
    
    m.obj = pyo.Objective(expr=m.makespan, sense=pyo.minimize)
    
    # --- F. Solve with MindtPy ---
    solver = SolverFactory('mindtpy')
    
    print("Starting MINLP Solver (This may take time)...")
    try:
        # Using 'oa' (Outer Approximation) is generally good for convex NLPs
        # time_limit ensures it doesn't hang forever on tutorial
        results = solver.solve(m, mip_solver='glpk', nlp_solver='ipopt', tee=debug, time_limit=600)
    except Exception as e:
        print(f"Solver Error: {e}")
        return pd.DataFrame()

    # --- G. Extract Results ---
    res = []
    for i in m.I:
        res.append({
            'aircraft_id': i,
            'optimized_d_i': pyo.value(m.d[i]),
            'arrival_time': pyo.value(m.t[i]),
            'v_L': pyo.value(m.v_L[i]),
            'v_theta': pyo.value(m.v_theta[i]),
            'v_f': pyo.value(m.v_f[i]),
            'entry_time': p_tau[i]
        })
        
    return pd.DataFrame(res).sort_values('arrival_time')

# --- Run ---
# Note: Ensure you have 'glpk' installed (conda install glpk)
# If not, this cell will throw an error.
df_seq_results = solve_sequencing_minlp(df_arrivals, debug=True)

if not df_seq_results.empty:
    print("\nOptimal Sequence Found:")
    # Calculate the sequence shifts
    df_seq_results['seq_rank'] = range(1, len(df_seq_results)+1)
    df_seq_results['original_id_rank'] = df_seq_results['aircraft_id'].rank()
    
    display(df_seq_results[['seq_rank', 'aircraft_id', 'entry_time', 'arrival_time', 'v_L', 'v_theta', 'v_f']])
    
    # Did the order change?
    if not df_seq_results['seq_rank'].equals(df_seq_results['original_id_rank']):
        print("NOTICE: The solver re-sequenced the aircraft to improve efficiency!")
    else:
        print("NOTICE: FCFS was found to be the optimal sequence.")

Starting MindtPy version 1.0.0 using OA algorithm
iteration_limit: 50
stalling_limit: 15
time_limit: 600
strategy: OA
add_regularization: None
call_after_main_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x7b81da34f2b0>
call_before_subproblem_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x7b81da34f2e0>
call_after_subproblem_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x7b81da34f310>
call_after_subproblem_feasible: <pyomo.contrib.gdpopt.util._DoNothing object at 0x7b81da34f340>
tee: true
logger: <Logger pyomo.contrib.mindtpy (INFO)>
logging_level: 20
integer_to_binary: false
add_no_good_cuts: false
use_tabu_list: false
single_tree: false
solution_pool: false
num_solution_iteration: 5
cycling_check: true
feasibility_norm: L_infinity
differentiate_mode: reverse_symbolic
use_mcpp: false
calculate_dual_at_solution: false
use_fbbt: false
use_dual_bound: true
partition_obj_nonlinear_terms: true
quadratic_strategy: 0
move_objective: false
add_cuts_at_incumbent: f

Solving Optimal Sequence for 40 aircraft...
Starting MINLP Solver (This may take time)...


Original model has 1720 constraints (40 nonlinear) and 0 disjunctions, with 981 variables, of which 780 are binary, 0 are integer, and 201 are continuous.
rNLP is the initial strategy being used.

 Iteration | Subproblem Type | Objective Value | Primal Bound |   Dual Bound |   Gap   | Time(s)



deprecated. Either specify deactivated Blocks as targets to activate them if
transforming them is the desired behavior.  (deprecated in 6.9.3) (called from
/home/yp6443/miniconda3/envs/tracon/lib/python3.10/site-
packages/pyomo/core/base/transformation.py:77)


KeyboardInterrupt: 

### with slack variables

In [ ]:
# ==========================================
# 9. ROBUST MINLP SEQUENCING
#    Variables: d_i, Speeds (Continuous), Sequence (Binary), Slack
# ==========================================

def solve_sequencing_minlp_robust(df_arrivals, debug=False):
    """
    Setup 3: Co-optimize Sequence, Speed (Continuous), and d_i.
    Includes Slack variables to prevent solver failure on bad sequences.
    """
    
    # --- LIMIT DATA SIZE ---
    # Sequencing is Factorial complexity (N!). 
    # Start with 5-7 aircraft. 10+ requires commercial solvers (Gurobi).
    df_subset = df_arrivals.head(5).copy() 
    print(f"Optimizing Sequence for first {len(df_subset)} aircraft...")
    
    m = pyo.ConcreteModel()
    
    # --- A. Sets & Parameters ---
    flight_ids = df_subset['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Pairs (i, j) for sequencing logic (i < j to avoid duplicates)
    m.Pairs = pyo.Set(initialize=[(i, j) for i in flight_ids for j in flight_ids if i < j])
    
    p_tau = df_subset.set_index('aircraft_id')['entry_time'].to_dict()
    p_x = df_subset.set_index('aircraft_id')['x_entry'].to_dict()
    p_y = df_subset.set_index('aircraft_id')['y_entry'].to_dict()
    p_north = df_subset.set_index('aircraft_id')['is_north'].to_dict()
    p_long = df_subset.set_index('aircraft_id')['long_arc'].to_dict()
    
    r = R_TURN_NM
    x_faf, y_faf = processed_fixes['VINII']['x'], processed_fixes['VINII']['y']
    
    # Big-M: Sufficiently large to turn off constraints (e.g., 1 hour)
    M = 3600.0 

    # --- B. Decision Variables ---
    
    # 1. Physics Variables
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0, 20000), initialize=lambda m, i: p_tau[i] + 600)
    
    # 2. Continuous Speed Variables (Setup 2.3 logic)
    m.v_L = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(180, 250), initialize=210)
    m.v_theta = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=160)
    m.v_f = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=140)
    
    # 3. Sequencing Binary: y[i,j]=1 if i before j
    m.y = pyo.Var(m.Pairs, domain=pyo.Binary)
    
    # 4. Pairwise Slack: Allows violation of separation for specific pairs
    # If the solver picks a bad sequence, it uses this slack variable.
    m.slack = pyo.Var(m.Pairs, domain=pyo.NonNegativeReals, initialize=0.0)
    
    # 5. Makespan (Scalar)
    m.makespan = pyo.Var(domain=pyo.NonNegativeReals, initialize=p_tau[flight_ids[-1]] + 1000)

    # --- C. Physics Constraints (Safe Geometry) ---
    def physics_rule(m, i):
        y_off = r if p_north[i] else -r
        Cx, Cy = x_faf - m.d[i], y_faf + y_off
        
        # Safe Geometry (Prevent sqrt crash)
        d0_sq = (p_x[i]-Cx)**2 + (p_y[i]-Cy)**2
        d0_p_sq = (p_x[i]-(x_faf-m.d[i]))**2 + (p_y[i]-y_faf)**2
        
        safe_term = pyo.Expr_if(d0_sq - r**2 > 1e-6, d0_sq - r**2, 0.0)
        d_L = pyo.sqrt(safe_term + 1e-6)
        
        d0 = pyo.sqrt(d0_sq + 1e-6)
        theta1 = pyo.acos(r/d0)
        theta2 = pyo.acos((r**2 + d0_sq - d0_p_sq)/(2*r*d0))
        theta_rad = (2*3.14159 - (theta1+theta2)) if p_long[i] else (theta2-theta1)
        
        d_theta = r * theta_rad
        d_final = m.d[i]

        t_L = (d_L / m.v_L[i]) * 3600
        t_turn = (d_theta / m.v_theta[i]) * 3600
        t_final = (d_final / m.v_f[i]) * 3600
        
        return m.t[i] == p_tau[i] + t_L + t_turn + t_final
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # Speed Monotonicity
    m.c_speed1 = pyo.Constraint(m.I, rule=lambda m, i: m.v_L[i] >= m.v_theta[i])
    m.c_speed2 = pyo.Constraint(m.I, rule=lambda m, i: m.v_theta[i] >= m.v_f[i])

    # --- D. Big-M Sequencing with Slack ---
    
    # Case 1: i comes before j (y=1)
    # Constraint: t[j] >= t[i] + Sep - Slack - BigM*(0)
    def seq_rule_1(m, i, j):
        return m.t[j] >= m.t[i] + T_SEP_SEC - m.slack[i, j] - M * (1 - m.y[i, j])
    m.c_seq_1 = pyo.Constraint(m.Pairs, rule=seq_rule_1)
    
    # Case 2: j comes before i (y=0)
    # Constraint: t[i] >= t[j] + Sep - Slack - BigM*(0)
    # Note: We use the SAME slack variable for the pair (i,j) to save memory
    def seq_rule_2(m, i, j):
        return m.t[i] >= m.t[j] + T_SEP_SEC - m.slack[i, j] - M * (m.y[i, j])
    m.c_seq_2 = pyo.Constraint(m.Pairs, rule=seq_rule_2)

    # --- E. Objective ---
    # Minimize Makespan + Heavy Penalty for using Slack
    def makespan_def(m, i):
        return m.makespan >= m.t[i]
    m.c_makespan = pyo.Constraint(m.I, rule=makespan_def)
    
    # Penalty = 10,000. 
    # Logic: It is much better to delay the makespan by 1 hour than to have 1 second of safety violation.
    m.obj = pyo.Objective(expr=m.makespan + 10000 * sum(m.slack[i, j] for i, j in m.Pairs), sense=pyo.minimize)
    
    # --- F. Solve ---
    print("Running MindtPy (Sequencing)...")
    solver = SolverFactory('mindtpy')
    
    try:
        # Strategy 'oa' (Outer Approximation) usually works best for these physics problems
        # time_limit is important so it doesn't hang indefinitely
        results = solver.solve(m, mip_solver='glpk', nlp_solver='ipopt', 
                               tee=debug, 
                               strategy='oa',
                               time_limit=600) 
    except Exception as e:
        print(f"Solver Error: {e}")
        return pd.DataFrame()

    # --- G. Extract ---
    res = []
    for i in m.I:
        res.append({
            'aircraft_id': i,
            'entry_time': p_tau[i],
            'optimized_d_i': pyo.value(m.d[i]),
            'arrival_time': pyo.value(m.t[i]),
            'v_L': pyo.value(m.v_L[i]),
            'v_f': pyo.value(m.v_f[i])
        })
    
    # Check total slack used
    total_slack = sum(pyo.value(m.slack[i, j]) for i, j in m.Pairs)
    print(f"\nTotal Sequencing Slack Used: {total_slack:.2f} s")
    if total_slack > 0.1:
        print("WARNING: The solver could not find a strictly valid sequence (capacity overloaded).")
        
    return pd.DataFrame(res).sort_values('arrival_time')

# --- Run ---
# Note: Ensure you have 'glpk' installed (conda install glpk)
# If not, this cell will throw an error.
df_seq_results = solve_sequencing_minlp(df_arrivals, debug=True)

if not df_seq_results.empty:
    print("\nOptimal Sequence Found:")
    # Calculate the sequence shifts
    df_seq_results['seq_rank'] = range(1, len(df_seq_results)+1)
    df_seq_results['original_id_rank'] = df_seq_results['aircraft_id'].rank()
    
    display(df_seq_results[['seq_rank', 'aircraft_id', 'entry_time', 'arrival_time', 'v_L', 'v_theta', 'v_f']])
    
    # Did the order change?
    if not df_seq_results['seq_rank'].equals(df_seq_results['original_id_rank']):
        print("NOTICE: The solver re-sequenced the aircraft to improve efficiency!")
    else:
        print("NOTICE: FCFS was found to be the optimal sequence.")

Starting MindtPy version 1.0.0 using OA algorithm
iteration_limit: 50
stalling_limit: 15
time_limit: 600
strategy: OA
add_regularization: None
call_after_main_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x7b81da34f2b0>
call_before_subproblem_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x7b81da34f2e0>
call_after_subproblem_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x7b81da34f310>
call_after_subproblem_feasible: <pyomo.contrib.gdpopt.util._DoNothing object at 0x7b81da34f340>
tee: true
logger: <Logger pyomo.contrib.mindtpy (INFO)>
logging_level: 20
integer_to_binary: false
add_no_good_cuts: false
use_tabu_list: false
single_tree: false
solution_pool: false
num_solution_iteration: 5
cycling_check: true
feasibility_norm: L_infinity
differentiate_mode: reverse_symbolic
use_mcpp: false
calculate_dual_at_solution: false
use_fbbt: false
use_dual_bound: true
partition_obj_nonlinear_terms: true
quadratic_strategy: 0
move_objective: false
add_cuts_at_incumbent: f

Solving Optimal Sequence for 40 aircraft...
Starting MINLP Solver (This may take time)...


KeyboardInterrupt: 

## Try the time-window decomposition approach
Instead of solving everything at once, we split the problem into two physically coupled steps:
- Step 1: Pre-calculate Feasibility Windows (The Physics)For each aircraft $i$, calculate the Earliest Possible Arrival Time ($E_i$) by flying the shortest path at maximum speed.Calculate the Latest Possible Arrival Time ($L_i$) by flying the longest path at minimum speed.Now, we know that aircraft $i$ can physically arrive anywhere in the interval $[E_i, L_i]$.
- Step 2: Solve the Sequencing (The Scheduling MILP)This becomes a pure Linear Programming problem (MILP). No square roots. No geometry.We just determine arrival times $t_i$ such that $E_i \le t_i \le L_i$ and everyone is separated by 64 seconds.This solves in milliseconds.
- Step 3: Trajectory Recovery (Post-processing)Once the MILP gives us the optimal time $t^*_i$, we simply find the specific $d_i$ and speed $v$ that achieves that time.

In [10]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# ==========================================
# PHASE 1: FEASIBILITY WINDOWS
# ==========================================

def get_flight_duration(d_i, v_L, v_theta, v_f, row, r, x_faf, y_faf):
    """Calculates flight time for specific control inputs."""
    y_off = r if row['is_north'] else -r
    Cx, Cy = x_faf - d_i, y_faf + y_off
    
    d0_sq = (row['x_entry']-Cx)**2 + (row['y_entry']-Cy)**2
    if d0_sq <= r**2 + 1e-4: return 99999.0 # Geometry Safety
    
    d_L = np.sqrt(d0_sq - r**2)
    d0 = np.sqrt(d0_sq)
    d0_p_sq = (row['x_entry']-(x_faf-d_i))**2 + (row['y_entry']-y_faf)**2
    
    # Clip for arccos safety
    term1 = np.clip(r/d0, -1.0, 1.0)
    term2 = np.clip((r**2 + d0_sq - d0_p_sq)/(2*r*d0), -1.0, 1.0)
    
    theta1 = np.arccos(term1)
    theta2 = np.arccos(term2)
    theta_rad = (2*np.pi - (theta1+theta2)) if row['long_arc'] else (theta2-theta1)
    
    # Total time
    dist_turn = r * theta_rad
    return (d_L/v_L + dist_turn/v_theta + d_i/v_f) * 3600.0

def calculate_windows(df):
    """Generates [Earliest, Latest] feasible arrival times."""
    r = R_TURN_NM
    x_faf, y_faf = processed_fixes['VINII']['x'], processed_fixes['VINII']['y']
    res = []
    
    for idx, row in df.iterrows():
        # Min Time: Max Speed (250/200/200), Min Path (d=0)
        t_min = get_flight_duration(0.0, 250.0, 200.0, 200.0, row, r, x_faf, y_faf)
        # Max Time: Min Speed (180/130/130), Max Path (d=20)
        t_max = get_flight_duration(20.0, 180.0, 130.0, 130.0, row, r, x_faf, y_faf)
        
        res.append({
            'aircraft_id': row['aircraft_id'],
            'entry_time': row['entry_time'],
            'min_arrival': row['entry_time'] + t_min,
            'max_arrival': row['entry_time'] + t_max
        })
    return pd.DataFrame(res)

# Run Phase 1
df_windows = calculate_windows(df_arrivals)
print("Windows Calculated.")

Windows Calculated.


In [11]:
# ==========================================
# PHASE 2: ROLLING HORIZON MILP
# ==========================================

def solve_linear_window(df_subset, prev_committed_time=None):
    """Solves sequence for a small window of aircraft (Linear MILP)."""
    m = pyo.ConcreteModel()
    
    ids = df_subset['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=ids)
    # Pairwise logic for this small window only
    m.Pairs = pyo.Set(initialize=[(i, j) for i in ids for j in ids if i < j])
    
    p_min = df_subset.set_index('aircraft_id')['min_arrival'].to_dict()
    p_max = df_subset.set_index('aircraft_id')['max_arrival'].to_dict()
    
    # Variables
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals)
    m.y = pyo.Var(m.Pairs, domain=pyo.Binary) 
    m.slack = pyo.Var(m.Pairs, domain=pyo.NonNegativeReals, initialize=0.0)
    m.boundary_slack = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=0.0)
    
    # Dynamic Big-M: Must be larger than the max possible timestamp spread
    M = 50000.0 

    # --- Constraints ---
    
    # 1. Physics Windows
    def window_rule(m, i):
        return (p_min[i], m.t[i], p_max[i])
    m.c_window = pyo.Constraint(m.I, rule=window_rule)
    
    # 2. Separation Logic
    def seq_1(m, i, j):
        return m.t[j] >= m.t[i] + T_SEP_SEC - m.slack[i,j] - M*(1-m.y[i,j])
    m.c_seq_1 = pyo.Constraint(m.Pairs, rule=seq_1)

    def seq_2(m, i, j):
        return m.t[i] >= m.t[j] + T_SEP_SEC - m.slack[i,j] - M*(m.y[i,j])
    m.c_seq_2 = pyo.Constraint(m.Pairs, rule=seq_2)
    
    # 3. Boundary Constraint (Link to previous window)
    if prev_committed_time is not None:
        def boundary_rule(m, i):
            # All planes in this new window must be after the last committed plane
            return m.t[i] >= prev_committed_time + T_SEP_SEC - m.boundary_slack[i]
        m.c_boundary = pyo.Constraint(m.I, rule=boundary_rule)

    # Objective: Minimize Sum of Arrival Times (Efficiency) + Slack Penalties
    obj_expr = sum(m.t[i] for i in m.I) \
             + 100000 * sum(m.slack[i,j] for i,j in m.Pairs) \
             + 200000 * sum(m.boundary_slack[i] for i in m.I)
             
    m.obj = pyo.Objective(expr=obj_expr, sense=pyo.minimize)
    
    # Solve (GLPK is fast for small MILPs)
    solver = SolverFactory('glpk')
    solver.solve(m)
    
    res = []
    for i in m.I:
        res.append({
            'aircraft_id': i,
            'target_time': pyo.value(m.t[i])
        })
    return pd.DataFrame(res).sort_values('target_time')

def run_sequencing_pipeline(df_wins):
    # Sort initially by Earliest Feasible Time
    df_pool = df_wins.sort_values('min_arrival').copy()
    
    WINDOW_SIZE = 8   # Sequence 8 planes at a time
    STEP_SIZE = 4     # Commit 4, slide window by 4
    
    final_schedule = []
    last_committed_time = None
    
    total_planes = len(df_pool)
    i = 0
    
    print(f"Sequencing {total_planes} aircraft (Rolling Horizon)...")
    
    while i < total_planes:
        # Define Window
        subset = df_pool.iloc[i : min(i + WINDOW_SIZE, total_planes)].copy()
        
        # Solve Window
        # print(f"  > Window {i} to {i+len(subset)}")
        df_res = solve_linear_window(subset, prev_committed_time=last_committed_time)
        
        # Commit the first 'STEP_SIZE' planes from the result
        # (Or all of them if we are at the end)
        num_to_commit = min(STEP_SIZE, len(df_res))
        
        # We take the FIRST 'num_to_commit' planes as ordered by the solver
        for k in range(num_to_commit):
            committed_plane = df_res.iloc[k]
            final_schedule.append(committed_plane.to_dict())
            last_committed_time = committed_plane['target_time']
            
        i += num_to_commit
        
    return pd.DataFrame(final_schedule)

# Run Phase 2
df_targets = run_sequencing_pipeline(df_windows)
print("Optimal Schedule Generated.")
display(df_targets)

Sequencing 40 aircraft (Rolling Horizon)...
Optimal Schedule Generated.


,aircraft_id,target_time
0,1.0,561.926255
1,2.0,714.113929
2,4.0,1048.586447
3,3.0,1186.486694
4,7.0,1252.486694
5,5.0,1420.074456
6,9.0,1495.724137
7,6.0,1561.724137
8,10.0,1627.724137
9,8.0,1771.160264


In [12]:
# ==========================================
# PHASE 3: RECOVER TRAJECTORY & METRICS
# ==========================================

def recover_trajectories_with_metrics(df_original, df_targs):
    """
    1. Finds optimal d_i and speeds to match the target time.
    2. Sorts by Arrival Time to determine the final sequence.
    3. Calculates separation based on the actual landing order.
    """
    combined = pd.merge(df_original, df_targs, on='aircraft_id')
    results = []
    
    r = R_TURN_NM
    x_faf, y_faf = processed_fixes['VINII']['x'], processed_fixes['VINII']['y']
    
    print(f"Recovering trajectories for {len(combined)} aircraft...")
    
    # --- Optimization Loop ---
    for idx, row in combined.iterrows():
        m = pyo.ConcreteModel()
        
        # Variables
        m.d = pyo.Var(domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=1.0)
        m.v_L = pyo.Var(domain=pyo.NonNegativeReals, bounds=(180, 250), initialize=210)
        m.v_theta = pyo.Var(domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=160)
        m.v_f = pyo.Var(domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=140)
        
        # Physics Rule
        def physics_rule(m):
            y_off = r if row['is_north'] else -r
            Cx, Cy = x_faf - m.d, y_faf + y_off
            
            d0_sq = (row['x_entry']-Cx)**2 + (row['y_entry']-Cy)**2
            d0_p_sq = (row['x_entry']-(x_faf-m.d))**2 + (row['y_entry']-y_faf)**2
            
            d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
            d0 = pyo.sqrt(d0_sq + 1e-6)
            
            theta1 = pyo.acos(r/d0)
            theta2 = pyo.acos((r**2 + d0_sq - d0_p_sq)/(2*r*d0))
            theta_rad = (2*3.14159 - (theta1+theta2)) if row['long_arc'] else (theta2-theta1)
            
            t_flight = (d_L/m.v_L + (r*theta_rad)/m.v_theta + m.d/m.v_f) * 3600
            
            # Constraint: Match Target Time
            return row['entry_time'] + t_flight == row['target_time']
            
        m.c_phys = pyo.Constraint(rule=physics_rule)
        
        # Monotonicity
        m.c_mono1 = pyo.Constraint(expr=m.v_L >= m.v_theta)
        m.c_mono2 = pyo.Constraint(expr=m.v_theta >= m.v_f)
        
        # Objective: Efficiency
        m.obj = pyo.Objective(expr=m.d - 0.001*(m.v_L + m.v_theta + m.v_f), sense=pyo.minimize)
        
        solver = SolverFactory('ipopt')
        solver.options['print_level'] = 0
        solver.solve(m)
        
        results.append({
            'entry_id': row['aircraft_id'], # Original Entry ID
            'entry_time': row['entry_time'],
            'arrival_time': row['target_time'],
            'd_i': pyo.value(m.d),
            'v_L': pyo.value(m.v_L),
            'v_theta': pyo.value(m.v_theta),
            'v_f': pyo.value(m.v_f)
        })
        
    # --- Post-Processing for Metrics ---
    df_res = pd.DataFrame(results)
    
    # 1. Sort by Actual Arrival Time (This defines the new sequence)
    df_res = df_res.sort_values('arrival_time').reset_index(drop=True)
    
    # 2. Assign Landing Rank (Current ID reaching FAF)
    df_res['landing_rank'] = df_res.index + 1
    
    # 3. Calculate Separation based on the NEW order
    # (Current plane time - Previous plane time)
    df_res['prev_arrival'] = df_res['arrival_time'].shift(1)
    df_res['separation'] = df_res['arrival_time'] - df_res['prev_arrival']
    
    # Fill NaN for the first aircraft (separation is N/A or infinite)
    df_res['separation'] = df_res['separation'].fillna(0.0)
    
    # 4. Calculate Transit Time
    df_res['transit_time'] = df_res['arrival_time'] - df_res['entry_time']

    return df_res

# --- Execute & Display ---
df_final = recover_trajectories_with_metrics(df_arrivals, df_targets)

# Display columns requested: Entry ID, Landing Rank, Arrival Time, Separation
cols_to_show = ['landing_rank', 'entry_id', 'entry_time', 'arrival_time', 'separation', 'd_i', 'v_L', 'v_theta', 'v_f']

print("\nFinal Optimized Schedule (Sorted by Landing Order):")
display(df_final[cols_to_show])

# Verification
min_sep = df_final.iloc[1:]['separation'].min() # Skip first row
print(f"\nMinimum Separation Achieved: {min_sep:.2f} s")
if min_sep >= T_SEP_SEC - 0.1:
    print(f"SUCCESS: Separation requirements met (target: {T_SEP_SEC}s).")
else:
    print(f"WARNING: Separation violation detected (target: {T_SEP_SEC}s).")

Recovering trajectories for 40 aircraft...

Final Optimized Schedule (Sorted by Landing Order):


,landing_rank,entry_id,entry_time,arrival_time,separation,d_i,v_L,v_theta,v_f
0,1,1,178.914341,561.926255,0.000000,-9.774867e-09,249.999999,200.000002,199.999997
1,2,2,331.102014,714.113929,152.187674,-9.774867e-09,249.999999,200.000002,199.999997
2,3,4,665.574532,1048.586447,334.472518,-9.774866e-09,249.999999,200.000002,199.999997
3,4,3,431.010455,1186.486694,137.900247,-7.447827e-09,249.999674,199.999999,199.999995
4,5,7,844.508337,1252.486694,66.000000,-9.083016e-09,241.011732,200.000001,200.000000
5,6,5,710.879026,1420.074456,167.587763,-7.444807e-09,249.999649,199.999999,199.999995
6,7,9,1098.527750,1495.724137,75.649680,-9.759397e-09,249.999999,200.000002,199.999997
7,8,6,764.391897,1561.724137,66.000000,-7.453571e-09,228.231101,199.999999,199.999995
8,9,10,1157.370978,1627.724137,66.000000,-7.464581e-09,191.732450,191.732448,191.732446
9,10,8,1061.964834,1771.160264,143.436127,-7.444807e-09,249.999649,199.999999,199.999995



Minimum Separation Achieved: 66.00 s
SUCCESS: Separation requirements met (target: 66.0s).


### Intervaled Speeds with Decomposition Method for the Non-Convex Mixed Integer Nonlinear Programming problem. 

In [13]:
# ==========================================
# PHASE 3: ONE-SIDED ROBUST RECOVERY
# Strategy: "Better Late Than Early"
# ==========================================

def recover_trajectories_discrete_safe(df_original, df_targs):
    combined = pd.merge(df_original, df_targs, on='aircraft_id')
    results = []
    
    r = R_TURN_NM
    x_faf, y_faf = processed_fixes['VINII']['x'], processed_fixes['VINII']['y']
    
    print(f"Recovering trajectories (Safe Mode) for {len(combined)} aircraft...")
    
    for idx, row in combined.iterrows():
        # ---------------------------------------------------------
        # STEP 1: CONTINUOUS SOLVE (Same as before)
        # ---------------------------------------------------------
        m = pyo.ConcreteModel()
        m.d = pyo.Var(domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=2.0)
        m.v_L = pyo.Var(domain=pyo.NonNegativeReals, bounds=(180, 250), initialize=210)
        m.v_theta = pyo.Var(domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=160)
        m.v_f = pyo.Var(domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=140)
        
        # Helper for physics
        def physics_calc(m, vL, vT, vF):
            y_off = r if row['is_north'] else -r
            Cx, Cy = x_faf - m.d, y_faf + y_off
            d0_sq = (row['x_entry']-Cx)**2 + (row['y_entry']-Cy)**2
            d0_p_sq = (row['x_entry']-(x_faf-m.d))**2 + (row['y_entry']-y_faf)**2
            
            d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
            d0 = pyo.sqrt(d0_sq + 1e-6)
            
            theta1 = pyo.acos(r/d0)
            theta2 = pyo.acos((r**2 + d0_sq - d0_p_sq)/(2*r*d0))
            theta_rad = (2*3.14159 - (theta1+theta2)) if row['long_arc'] else (theta2-theta1)
            
            return (d_L/vL + (r*theta_rad)/vT + m.d/vF) * 3600

        # Constraint: Match Target exactly
        m.c_phys = pyo.Constraint(expr=row['entry_time'] + physics_calc(m, m.v_L, m.v_theta, m.v_f) == row['target_time'])
        m.c_mono1 = pyo.Constraint(expr=m.v_L >= m.v_theta)
        m.c_mono2 = pyo.Constraint(expr=m.v_theta >= m.v_f)
        m.obj = pyo.Objective(expr=m.d - 0.001*(m.v_L + m.v_theta + m.v_f), sense=pyo.minimize)
        
        solver = SolverFactory('ipopt')
        solver.options['print_level'] = 0
        try: solver.solve(m)
        except: pass
        
        # ---------------------------------------------------------
        # STEP 2: ROUNDING (Same as before)
        # ---------------------------------------------------------
        def round10(x): return 10.0 * round(pyo.value(x) / 10.0)
        d_vL = max(180, min(250, round10(m.v_L)))
        d_vT = max(130, min(200, round10(m.v_theta)))
        d_vF = max(130, min(200, round10(m.v_f)))
        
        if d_vT > d_vL: d_vT = d_vL
        if d_vF > d_vT: d_vF = d_vT
        fixed_speeds = (d_vL, d_vT, d_vF)

        # ---------------------------------------------------------
        # STEP 3: ONE-SIDED REPAIR (Better Late Than Early)
        # ---------------------------------------------------------
        m2 = pyo.ConcreteModel()
        m2.d = pyo.Var(domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=pyo.value(m.d))
        
        # We add a slack variable to prevent crashing, but we penalize it heavily
        m2.slack_early = pyo.Var(domain=pyo.NonNegativeReals, initialize=0.0)
        
        # Actual Time Calculation
        current_time = row['entry_time'] + physics_calc(m2, fixed_speeds[0], fixed_speeds[1], fixed_speeds[2])
        
        # CONSTRAINT: Actual Time must be >= Target Time (minus slack)
        # This forces the solver to increase d_i (delay) to satisfy the inequality
        m2.c_safe = pyo.Constraint(expr=current_time >= row['target_time'] - m2.slack_early)
        
        # OBJECTIVE:
        # 1. Minimize Slack * 1,000,000 (Primary: Do not arrive early!)
        # 2. Minimize (Actual - Target) (Secondary: Don't be too late, just enough to be safe)
        m2.obj = pyo.Objective(expr=1000000*m2.slack_early + (current_time - row['target_time']), sense=pyo.minimize)
        
        solver.solve(m2)
        
        actual_arrival = pyo.value(row['entry_time'] + physics_calc(m2, fixed_speeds[0], fixed_speeds[1], fixed_speeds[2]))

        results.append({
            'entry_id': row['aircraft_id'],
            'entry_time': row['entry_time'],
            'target_time': row['target_time'],
            'arrival_time': actual_arrival,
            'd_i': pyo.value(m2.d),
            'v_L': fixed_speeds[0],
            'v_theta': fixed_speeds[1],
            'v_f': fixed_speeds[2]
        })
        
    # --- Metrics ---
    df_res = pd.DataFrame(results)
    df_res = df_res.sort_values('arrival_time').reset_index(drop=True)
    df_res['landing_rank'] = df_res.index + 1
    df_res['prev_arrival'] = df_res['arrival_time'].shift(1)
    df_res['separation'] = df_res['arrival_time'] - df_res['prev_arrival']
    df_res['separation'] = df_res['separation'].fillna(0.0)
    
    return df_res

# --- Execute ---
# Recalculate using the Safe Mode
df_final_safe = recover_trajectories_discrete_safe(df_arrivals, df_targets)

print("\nFinal Optimized Schedule (Safe Mode):")
cols = ['landing_rank', 'entry_id', 'target_time', 'arrival_time', 'separation', 'd_i', 'v_L', 'v_theta', 'v_f']
display(df_final_safe[cols])

# Check robustness
min_sep = df_final_safe.iloc[1:]['separation'].min()
print(f"\nMinimum Separation: {min_sep:.2f}s")

Recovering trajectories (Safe Mode) for 40 aircraft...
WARNING (W1001): Setting Var 'd' to a value `-9.757839877273915e-09` (float)
not in domain NonNegativeReals.
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1001
WARNING (W1001): Setting Var 'd' to a value `-9.757839923338677e-09` (float)
not in domain NonNegativeReals.
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1001
WARNING (W1001): Setting Var 'd' to a value `-7.447826961646329e-09` (float)
not in domain NonNegativeReals.
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1001
WARNING (W1001): Setting Var 'd' to a value `-9.757839777776581e-09` (float)
not in domain NonNegativeReals.
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1001
WARNING (W1001): Setting Var 'd' to a value `-7.444807255087497e-09` (float)
not in domain NonNegativeReals.
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1001
WARNING (W1001): Setting Var 'd' to a value `-7.45

,landing_rank,entry_id,target_time,arrival_time,separation,d_i,v_L,v_theta,v_f
0,1,1,561.926255,561.926311,0.000000,4.807155e-06,250.0,200.0,200.0
1,2,2,714.113929,714.113985,152.187674,4.807155e-06,250.0,200.0,200.0
2,3,4,1048.586447,1048.586503,334.472518,4.807155e-06,250.0,200.0,200.0
3,4,3,1186.486694,1186.486720,137.900217,1.859262e-05,250.0,200.0,200.0
4,5,7,1252.486694,1253.750920,67.264201,2.340905e-06,240.0,200.0,200.0
5,6,5,1420.074456,1420.074483,166.323563,1.885894e-05,250.0,200.0,200.0
6,7,9,1495.724137,1495.724193,75.649709,4.998385e-06,250.0,200.0,200.0
7,8,6,1561.724137,1561.724161,65.999968,1.106667e-01,230.0,200.0,200.0
8,9,10,1627.724137,1632.012890,70.288730,-9.994532e-09,190.0,190.0,190.0
9,10,8,1771.160264,1771.160290,139.147400,1.885894e-05,250.0,200.0,200.0



Minimum Separation: 55.52s


### Hard Separation Time Enforcement

In [14]:
# ==========================================
# PHASE 3: SEQUENTIAL DISCRETE RECOVERY
# Strategy: Propagate delays downstream to enforce hard safety.
# ==========================================

def recover_trajectories_sequential_hard(df_original, df_targs):
    # 1. Merge and Sort by Phase 2 Target Time
    combined = pd.merge(df_original, df_targs, on='aircraft_id')
    combined = combined.sort_values('target_time').reset_index(drop=True)
    
    results = []
    
    # Track the actual arrival of the previous plane
    # Initialize with -infinity so the first plane is unconstrained by separation
    last_actual_arrival = -99999.0 
    
    r = R_TURN_NM
    x_faf, y_faf = processed_fixes['VINII']['x'], processed_fixes['VINII']['y']
    
    print(f"Recovering trajectories sequentially for {len(combined)} aircraft...")
    
    for idx, row in combined.iterrows():
        
        # --- DYNAMIC TARGET UPDATE ---
        # The new hard target is the max of:
        # 1. The original optimal schedule (Phase 2)
        # 2. The previous plane's actual landing time + T_SEP_SEC
        hard_target_time = max(row['target_time'], last_actual_arrival + T_SEP_SEC)
        
        # ---------------------------------------------------------
        # STEP 1: CONTINUOUS SOLVE (Find Ideal Profile)
        # ---------------------------------------------------------
        m = pyo.ConcreteModel()
        m.d = pyo.Var(domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=2.0)
        m.v_L = pyo.Var(domain=pyo.NonNegativeReals, bounds=(180, 250), initialize=210)
        m.v_theta = pyo.Var(domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=160)
        m.v_f = pyo.Var(domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=140)
        
        def physics_calc(m, vL, vT, vF):
            y_off = r if row['is_north'] else -r
            Cx, Cy = x_faf - m.d, y_faf + y_off
            d0_sq = (row['x_entry']-Cx)**2 + (row['y_entry']-Cy)**2
            d0_p_sq = (row['x_entry']-(x_faf-m.d))**2 + (row['y_entry']-y_faf)**2
            
            d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
            d0 = pyo.sqrt(d0_sq + 1e-6)
            
            theta1 = pyo.acos(r/d0)
            theta2 = pyo.acos((r**2 + d0_sq - d0_p_sq)/(2*r*d0))
            theta_rad = (2*3.14159 - (theta1+theta2)) if row['long_arc'] else (theta2-theta1)
            
            return (d_L/vL + (r*theta_rad)/vT + m.d/vF) * 3600

        # Try to match the HARD target
        m.c_phys = pyo.Constraint(expr=row['entry_time'] + physics_calc(m, m.v_L, m.v_theta, m.v_f) == hard_target_time)
        m.c_mono1 = pyo.Constraint(expr=m.v_L >= m.v_theta)
        m.c_mono2 = pyo.Constraint(expr=m.v_theta >= m.v_f)
        
        # Efficiency Objective
        m.obj = pyo.Objective(expr=m.d - 0.001*(m.v_L + m.v_theta + m.v_f), sense=pyo.minimize)
        
        solver = SolverFactory('ipopt')
        solver.options['print_level'] = 0
        try: solver.solve(m)
        except: pass
        
        # ---------------------------------------------------------
        # STEP 2: ROUNDING SPEEDS
        # ---------------------------------------------------------
        def round10(x): return 10.0 * round(pyo.value(x) / 10.0)
        d_vL = max(180, min(250, round10(m.v_L)))
        d_vT = max(130, min(200, round10(m.v_theta)))
        d_vF = max(130, min(200, round10(m.v_f)))
        
        if d_vT > d_vL: d_vT = d_vL
        if d_vF > d_vT: d_vF = d_vT
        fixed_speeds = (d_vL, d_vT, d_vF)

        # ---------------------------------------------------------
        # STEP 3: REPAIR WITH HARD CONSTRAINT
        # ---------------------------------------------------------
        m2 = pyo.ConcreteModel()
        m2.d = pyo.Var(domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=pyo.value(m.d))
        
        arrival_expression = row['entry_time'] + physics_calc(m2, fixed_speeds[0], fixed_speeds[1], fixed_speeds[2])
        
        # HARD CONSTRAINT: Arrival must be >= Hard Target
        # This forces the plane to delay (increase d_i) if the discrete speeds are too fast
        m2.c_safety = pyo.Constraint(expr=arrival_expression >= hard_target_time)
        
        # OBJECTIVE: Be as close to the target as possible (minimize delay)
        m2.obj = pyo.Objective(expr=(arrival_expression - hard_target_time), sense=pyo.minimize)
        
        res_stat = solver.solve(m2)
        
        # ---------------------------------------------------------
        # STEP 4: UPDATE STATE
        # ---------------------------------------------------------
        actual_arrival = pyo.value(arrival_expression)
        
        # Update the 'last_arrival' for the next iteration
        last_actual_arrival = actual_arrival

        results.append({
            'entry_id': row['aircraft_id'],
            'entry_time': row['entry_time'],
            'target_time': row['target_time'], # Original Phase 2 plan
            'hard_target': hard_target_time,   # Updated safe constraint
            'arrival_time': actual_arrival,    # Final actual time
            'delay_added': actual_arrival - row['target_time'],
            'd_i': pyo.value(m2.d),
            'v_L': fixed_speeds[0],
            'v_theta': fixed_speeds[1],
            'v_f': fixed_speeds[2]
        })
        
    # --- Metrics ---
    df_res = pd.DataFrame(results)
    
    # Recalculate separation to prove it works
    df_res['prev_arrival'] = df_res['arrival_time'].shift(1)
    df_res['separation'] = df_res['arrival_time'] - df_res['prev_arrival']
    df_res['separation'] = df_res['separation'].fillna(0.0)
    df_res['landing_rank'] = df_res.index + 1
    return df_res

# --- Execute ---
# 1. Phase 1 (Feasibility Windows) - Same as before
df_windows = calculate_windows(df_arrivals)

# 2. Phase 2 (Linear Schedule) - Same as before
df_targets = run_sequencing_pipeline(df_windows) 

# 3. Phase 3 (Sequential Hard Constraint)
df_final_hard = recover_trajectories_sequential_hard(df_arrivals, df_targets)

# --- Verification ---
print("\nFinal Hard-Constraint Schedule:")
cols = ['entry_id', 'target_time', 'hard_target', 'arrival_time', 'separation', 'd_i', 'v_L', 'v_theta', 'v_f']
display(df_final_hard[cols])

min_sep = df_final_hard.iloc[1:]['separation'].min()
print(f"\nMinimum Separation Achieved: {min_sep:.4f} s")

if min_sep >= 63.999:
    print("SUCCESS: Hard safety constraints enforced.")
else:
    print("WARNING: Separation violation.")

Sequencing 40 aircraft (Rolling Horizon)...
Recovering trajectories sequentially for 40 aircraft...
WARNING (W1001): Setting Var 'd' to a value `-9.757839877273915e-09` (float)
not in domain NonNegativeReals.
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1001
WARNING (W1001): Setting Var 'd' to a value `-9.757839923338677e-09` (float)
not in domain NonNegativeReals.
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1001
WARNING (W1001): Setting Var 'd' to a value `-9.757839777776581e-09` (float)
not in domain NonNegativeReals.
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1001
WARNING (W1001): Setting Var 'd' to a value `-7.447826961646329e-09` (float)
not in domain NonNegativeReals.
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1001
WARNING (W1001): Setting Var 'd' to a value `-9.083016047942721e-09` (float)
not in domain NonNegativeReals.
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1001
WARNI

,entry_id,target_time,hard_target,arrival_time,separation,d_i,v_L,v_theta,v_f
0,1,561.926255,561.926255,561.926255,0.000000,-9.683441e-09,250.0,200.0,200.0
1,2,714.113929,714.113929,714.113929,152.187674,-9.683441e-09,250.0,200.0,200.0
2,4,1048.586447,1048.586447,1048.586447,334.472518,-9.683440e-09,250.0,200.0,200.0
3,3,1186.486694,1186.486694,1186.486693,137.900246,1.775129e-05,250.0,200.0,200.0
4,7,1252.486694,1252.486694,1253.750895,67.264203,-9.765187e-09,240.0,200.0,200.0
5,5,1420.074456,1420.074456,1420.074456,166.323561,1.800429e-05,250.0,200.0,200.0
6,9,1495.724137,1495.724137,1495.724137,75.649681,-9.684920e-09,250.0,200.0,200.0
7,6,1561.724137,1561.724137,1561.724136,65.999999,1.106659e-01,230.0,200.0,200.0
8,10,1627.724137,1627.724137,1632.012890,70.288755,-9.728292e-09,190.0,190.0,190.0
9,8,1771.160264,1771.160264,1771.160263,139.147373,1.800429e-05,250.0,200.0,200.0



Minimum Separation Achieved: 66.0000 s
SUCCESS: Hard safety constraints enforced.


## Setup 3 Visualizations

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import imageio
import os
import shutil

# ==========================================
# 1. CORRECTED TRAJECTORY ENGINE
# ==========================================

def get_aircraft_trajectory_points_corrected(row, x_faf, y_faf, r):
    """
    Reconstructs waypoints with CORRECT tangent geometry.
    North Centers (y > 0) -> CCW Turn -> Tangent Angle = Alpha + Beta
    South Centers (y < 0) -> CW Turn  -> Tangent Angle = Alpha - Beta
    """
    # 1. Inputs
    d_i = row['d_i']
    entry_x, entry_y = row['x_entry'], row['y_entry']
    is_north = row['is_north']
    is_long = row['long_arc']
    
    # Speeds (nm/s)
    vL_sec = row['v_L'] / 3600.0
    vT_sec = row['v_theta'] / 3600.0
    vF_sec = row['v_f'] / 3600.0
    
    # 2. Geometry Centers
    y_off = r if is_north else -r
    Cx, Cy = x_faf - d_i, y_faf + y_off
    
    # 3. Tangent Geometry
    # Vector from Center to Entry
    dx, dy = entry_x - Cx, entry_y - Cy
    d0 = np.sqrt(dx**2 + dy**2)
    
    # Alpha: Angle of vector Center->Entry
    alpha = np.arctan2(dy, dx)
    
    # Beta: The offset angle to the tangent point
    # Clamp to prevent domain error if d0 slightly < r due to numerical noise
    beta = np.arccos(np.clip(r / d0, -1.0, 1.0))
    
    # --- CRITICAL FIX HERE ---
    if is_north:
        # North Center = CCW Turn. 
        # Tangent must be "Left" of the incoming radius to support CCW flow.
        theta_start = alpha + beta 
    else:
        # South Center = CW Turn.
        # Tangent must be "Right" of the incoming radius to support CW flow.
        theta_start = alpha - beta
        
    start_x = Cx + r * np.cos(theta_start)
    start_y = Cy + r * np.sin(theta_start)
    
    # 4. End Point (Tangent to Final Leg)
    # The turn always ends at the vertical projection of Center onto Final Approach
    end_x, end_y = Cx, y_faf 
    
    # 5. Calculate Exact Sweep from Physics Engine (to match timing)
    d0_p_sq = (entry_x - end_x)**2 + (entry_y - y_faf)**2
    
    # Re-calculate lengths exactly as Optimization did
    d_L_len = np.sqrt(d0**2 - r**2)
    
    # Angle calculation for Sweep Magnitude
    term1 = np.clip(r/d0, -1, 1)
    term2 = np.clip((r**2 + d0**2 - d0_p_sq)/(2*r*d0), -1, 1)
    t1 = np.arccos(term1)
    t2 = np.arccos(term2)
    
    # Total radians to turn
    sweep_rad = (2*np.pi - (t1+t2)) if is_long else (t2-t1)
    
    # Determine Sweep Direction for interpolation
    # North (CCW) = Positive, South (CW) = Negative
    sweep_dir = 1.0 if is_north else -1.0
    
    d_theta_len = r * sweep_rad
    d_final_len = d_i
    
    # 6. Durations
    t_L = d_L_len / vL_sec
    t_turn = d_theta_len / vT_sec
    t_final = d_final_len / vF_sec
    
    return {
        'pts': [(entry_x, entry_y), (start_x, start_y), (end_x, end_y), (x_faf, y_faf)],
        'times': [row['entry_time'], row['entry_time']+t_L, row['entry_time']+t_L+t_turn, row['arrival_time']],
        'arc_params': (theta_start, sweep_rad, sweep_dir, Cx, Cy),
        'corner': row['corner']
    }

def interpolate_position_corrected(info, current_time):
    """
    Interpolates position (x, y) based on current time.
    """
    times = info['times']
    
    if current_time < times[0]: return None
    # Hide aircraft 15s after landing
    if current_time > times[3]: 
        return info['pts'][3] if current_time < times[3] + 15 else None
    
    pts = info['pts']
    
    # Segment 1: Tangent (Straight)
    if current_time <= times[1]:
        pct = (current_time - times[0]) / (times[1] - times[0])
        x = pts[0][0] + pct * (pts[1][0] - pts[0][0])
        y = pts[0][1] + pct * (pts[1][1] - pts[0][1])
        return x, y
        
    # Segment 2: Turn (Circular)
    if current_time <= times[2]:
        pct = (current_time - times[1]) / (times[2] - times[1])
        
        theta_start, sweep_rad, sweep_dir, cx, cy = info['arc_params']
        
        # Linearly interpolate angle
        current_angle = theta_start + (sweep_dir * sweep_rad * pct)
        
        x = cx + R_TURN_NM * np.cos(current_angle)
        y = cy + R_TURN_NM * np.sin(current_angle)
        return x, y
        
    # Segment 3: Final (Straight)
    if current_time <= times[3]:
        pct = (current_time - times[2]) / (times[3] - times[2])
        x = pts[2][0] + pct * (pts[3][0] - pts[2][0])
        y = pts[2][1] + pct * (pts[3][1] - pts[2][1])
        return x, y
        
    return None

# ==========================================
# 2. GENERATION LOOP
# ==========================================

def generate_corrected_gif(df_res, fps=0.1, speed_up=10):
    
    print("Pre-computing trajectories...")
    traj_db = {}
    r = R_TURN_NM
    x_faf, y_faf = processed_fixes['VINII']['x'], processed_fixes['VINII']['y']
    
    # Pre-compute geometries
    for idx, row in df_res.iterrows():
        traj_db[row['entry_id']] = get_aircraft_trajectory_points_corrected(row, x_faf, y_faf, r)
        
    t_start = df_res['entry_time'].min()
    t_end = df_res['arrival_time'].max() + 20
    dt = speed_up / fps # Simulation step per frame
    
    sim_times = np.arange(t_start, t_end, dt)
    print(f"Generating {len(sim_times)} frames...")
    
    filenames = []
    output_folder = 'sim_frames'
    if os.path.exists(output_folder): shutil.rmtree(output_folder)
    os.makedirs(output_folder)
    
    # Plot Limits
    all_x = df_res['x_entry'].tolist()
    all_y = df_res['y_entry'].tolist()
    pad = 5
    x_lim = (min(all_x)-pad, max(all_x)+pad)
    y_lim = (min(all_y)-pad, max(all_y)+pad)
    
    colors = {'NorthWest': 'crimson', 'NorthEast': 'royalblue', 'SouthEast': 'forestgreen', 'SouthWest': 'darkorange'}
    
    for i, t in enumerate(sim_times):
        fig, ax = plt.subplots(figsize=(10, 8), dpi=100)
        
        # Draw Background
        ax.plot(x_faf, y_faf, 'k^', markersize=8, zorder=5) # FAF
        ax.plot([x_faf, 0], [0, 0], 'k-', linewidth=2, alpha=0.5) # Runway Centerline
        ax.text(x_faf, -2, "FAF", ha='center', fontsize=8)

        ax.plot(0, 0, 'k*', markersize=12)
        ax.text(0, 0+1, "RWY 9L", ha='center')

        # Draw Trajectories (Background Traces)
        for eid, info in traj_db.items():
            # Only draw if active or recently finished
            if t > info['times'][0] and t < info['times'][3] + 30:
                pts = info['pts']
                theta_start, sweep_rad, sweep_dir, cx, cy = info['arc_params']
                
                # Draw Tangent Leg
                ax.plot([pts[0][0], pts[1][0]], [pts[0][1], pts[1][1]], 
                        color='gray', linestyle=':', linewidth=1, alpha=0.5)
                
                # Draw Final Leg
                ax.plot([pts[2][0], pts[3][0]], [pts[2][1], pts[3][1]], 
                        color='gray', linestyle=':', linewidth=1, alpha=0.5)
                
                # Draw Arc
                theta_draw = np.linspace(theta_start, theta_start + sweep_dir*sweep_rad, 30)
                arc_x = cx + r * np.cos(theta_draw)
                arc_y = cy + r * np.sin(theta_draw)
                ax.plot(arc_x, arc_y, color='gray', linestyle=':', linewidth=1, alpha=0.5)

        # Draw Aircraft
        active_count = 0
        for idx, row in df_res.iterrows():
            eid = row['entry_id']
            pos = interpolate_position_corrected(traj_db[eid], t)
            
            if pos:
                active_count += 1
                c = colors.get(row['corner'], 'black')
                
                # Draw Dot
                ax.plot(pos[0], pos[1], 'o', color=c, markersize=8, markeredgecolor='white', zorder=10)
                # Draw ID
                ax.text(pos[0]+0.8, pos[1]+0.8, f"{int(eid)}", fontsize=9, fontweight='bold', zorder=10)
                
                # Separation Line Visual
                my_rank = row['landing_rank']
                if my_rank > 1:
                    prev_row = df_res[df_res['landing_rank'] == my_rank - 1].iloc[0]
                    prev_pos = interpolate_position_corrected(traj_db[prev_row['entry_id']], t)
                    
                    if prev_pos:
                        dist = np.sqrt((pos[0]-prev_pos[0])**2 + (pos[1]-prev_pos[1])**2)
                        # Draw line to leader
                        if dist < 15: # Only show local links
                            col = 'red' if dist < 2.5 else 'green' # 2.5nm roughly 64s
                            width = 2 if dist < 2.5 else 1
                            ax.plot([pos[0], prev_pos[0]], [pos[1], prev_pos[1]], color=col, linewidth=width, alpha=0.6)

        ax.set_xlim(x_lim)
        ax.set_ylim(y_lim)
        ax.set_title(f"T = {t:.1f}s | Active: {active_count}")
        ax.set_aspect('equal')
        
        # Save
        fname = f"{output_folder}/frame_{i:04d}.png"
        plt.savefig(fname)
        filenames.append(fname)
        plt.close(fig)
        
        if i % 20 == 0: print(f"Frame {i}/{len(sim_times)}")
        
    print("Compiling GIF...")
    with imageio.get_writer('katl_final_setup3.gif', mode='I', fps=fps) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
    print("Saved: katl_final_corrected.gif")

# ==========================================
# RUN
# ==========================================
# 1. Ensure df_final_hard has geometry columns merged
cols_needed = ['x_entry', 'y_entry', 'corner', 'is_north', 'long_arc', 'v_L', 'v_theta', 'v_f']
df_viz = pd.merge(df_final_hard, df_arrivals[['aircraft_id', 'x_entry', 'y_entry', 'corner', 'is_north', 'long_arc']], 
                  left_on='entry_id', right_on='aircraft_id', suffixes=('', '_orig'))

# 2. Generate
generate_corrected_gif(df_viz, fps=10, speed_up=100)

Pre-computing trajectories...
Generating 403 frames...
Frame 0/403
Frame 20/403
Frame 40/403
Frame 60/403
Frame 80/403
Frame 100/403
Frame 120/403
Frame 140/403
Frame 160/403
Frame 180/403
Frame 200/403
Frame 220/403
Frame 240/403
Frame 260/403
Frame 280/403
Frame 300/403
Frame 320/403
Frame 340/403
Frame 360/403
Frame 380/403
Frame 400/403
Compiling GIF...


/tmp/ipykernel_2932662/3393031555.py:251: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


Saved: katl_final_corrected.gif
